In [1]:
# Importing dependencies

# !pip install transformers
# !pip install google-api-python client
# !pip install -U sentence-transformers
# !pip install faiss-gpu
# !pip install --upgrade bleu
# import bleu
# from bleu import list_bleu
# !python -m spacy download en_core_web_sm
# !python -m spacy download fr_core_news_sm
# !pip install laserembeddings
# !python -m laserembeddings download-models
# !pip install cython numpy
# !pip install benepar
# !pip install word2word
# !pip install pylcs
# !pip install google-cloud-translate
# !pip install google-auth==1.6.3
# pip install google-auth-oauthlib
#import benepar
import pylcs
from word2word import Word2word as w2w
import spacy
from google.cloud import translate
import torch
from torch import nn
from transformers import XLMRobertaModel, XLMRobertaTokenizer
from sentence_transformers import SentenceTransformer, util
import numpy as np
import faiss
import random
import re
import time
import csv
from googleapiclient.discovery import build
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk import tokenize
from nltk.translate.bleu_score import sentence_bleu
from nltk.tag import StanfordNERTagger
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('wordnet')
#benepar.download('benepar_fr')
#benepar.download('benepar_en2')
nltk.download('stopwords')
from bs4 import BeautifulSoup
import getpass
from collections import Counter

[nltk_data] Downloading package punkt to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /usr2/collab/agjones/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Checking for a GPU

class GPUError(Exception):
    pass

# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    
    if torch.cuda.get_device_name(0) == "Tesla K40m":
        raise GPUError("GPU Error: No compatible GPU found")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla V100-PCIE-16GB


In [3]:
# Building and configuring what we need for the multilingual sentence embeddings

# word_model = XLMRobertaModel.from_pretrained('xlm-roberta-base')
# word_tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
# word_model.cuda()

model_name = 'LaBSE'
sentence_model = SentenceTransformer(model_name)
sentence_model.cuda()

SentenceTransformer(
  (0): Transformer(
    (auto_model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(501153, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
      

In [4]:
# Functions needed for averaged word embeddings (= sentence embeddings)

# Tokenize with XLMRoBERTa
def word_tokenize(sentence):
    tokenized_sentence = torch.LongTensor([word_tokenizer.encode(sentence, add_special_tokens=False)]) #, add_special_tokens=True, padding='max_length', max_length=40, truncation=True)])
    tokenized_sentence = tokenized_sentence.to(device)
    return tokenized_sentence

# Embed with SBERT/LaBSE
# Returns SENTENCE embeddings of input sentences
def embed(sentences):
    sentence_embeddings = sentence_model.encode(sentences)
    return sentence_embeddings

# Embed with XLMRoBERTa
# Returns WORD embeddings of input sentences
def word_embed(tokenized_word):
    word_embedding = word_model.embeddings(tokenized_word)
    word_embedding = torch.mean(word_embedding, dim=1)
    return word_embedding.detach().cpu()
    

    # NB: Already dividing by the norm to create embedding!!
#     sentence_embedding = model.embeddings(tokenized_sentence)
#     #sentence_embedding = torch.mean(sentence_embedding, dim=1) # Using mean of hidden states as sentence embeddings
#     sentence_embedding = sentence_embedding[0][-1] # Using CLS token hidden state as sentence embedding
#     return sentence_embedding.detach().cpu()

# def getCosineSimilarity(embedding1, embedding2):
#     pass
# #     cos_sim_func = nn.CosineSimilarity(dim=0)
# #     cosine_similarity = cos_sim_func(embedding1, embedding2)
# #     return cosine_similarity.detach().cpu().numpy()

# def getAvgCosineSimilarity(sentence1, sentence2):
#     tokenized_sentence1 = tokenize_sentence(sentence1)
#     tokenized_sentence2 = tokenize_sentence(sentence2)
#     sentence1_embedding = embed(tokenized_sentence1)
#     sentence2_embedding = embed(tokenized_sentence2)
#     cosine_similarity = getCosineSimilarity(sentence1_embedding, sentence2_embedding)
#     return cosine_similarity

In [ ]:
# sent1_emb = embed(tokenize_sentence(sent1))
# sent2_emb = embed(tokenize_sentence(sent2))
# print(sent1_emb.shape)
# print(sent1_emb)
# print(sent2_emb)
# print(getCosineSimilarity(sent1_emb, sent2_emb))

In [ ]:
######################################################################
# PERFORMING SOME TESTS ON TRAINING DATA FROM THE BUCC'18 PARALLEL 
# SENTENCE EXTRACTION TASK
######################################################################

In [10]:
# Importing BUCC data

bucc_column_names = ['ID', 'Text']

# En/Fr sentences with IDs
english_sents = pd.read_csv('Data/fr-en.training_en.txt', names=bucc_column_names, delimiter="\t", quoting=csv.QUOTE_NONE)
french_sents = pd.read_csv('Data/fr-en.training_fr.txt', names=bucc_column_names, delimiter="\t", quoting=csv.QUOTE_NONE)

gold_column_names = ['fr_id', 'en_id']

# IDs for artificially-inserted gold-standard sentence pairs
gold_pairs = pd.read_csv('Data/fr-en.training_gold.csv', names=gold_column_names, error_bad_lines=False)
gold_pairs['fr_id'] = gold_pairs['fr_id'].map(lambda x: x.rstrip('\t'))

# Reading in the English training data that's been translated to French (using HuggingFace's 
# OPUS MT / Helsinki-NLP open-source NMT)
with open('Data/translated_english_sentences_all.txt', 'r') as f:
    transl_en_sents = f.read().splitlines()
    
fr_vocab = pd.read_csv('Data/fr_vocab.txt', names=['words'], encoding='latin_1', error_bad_lines=False)
fr_vocab = [word for word in fr_vocab['words']]

en_fr_dict = pd.read_csv('Dictionaries/en-fr_dict.txt', names=['entries'], error_bad_lines=False)

In [11]:
en_fr_dict = [entry for entry in en_fr_dict['entries']]
en_fr_dict_list = []
for entry in en_fr_dict:
    if type(entry)==str and len(entry.split())==2:
        en_fr_dict_list.append((entry.split()[0], entry.split()[1]))

In [12]:
start = time.time()
en_fr_dict_dict = {en_fr_dict_list[0][0]:[en_fr_dict_list[0][1]]}
for entry in en_fr_dict_list[1:]:
    if entry[0] in en_fr_dict_dict:
        en_fr_dict_dict[entry[0]].append(entry[1])
    else:
        en_fr_dict_dict[entry[0]] = [entry[1]]
end = time.time()
print(end-start)

0.21710562705993652


In [121]:
# transl = ""
# nstp = [word for word in nltk.word_tokenize(en_sents_only[1500]) if word not in stopwords.words('english')]
# for word in nstp:
#     if word in en_fr_dict_dict:
#         transl += en_fr_dict_dict[word][0] + " "
#     else:
#         transl += word + " "
# print(transl)

11 D , much manifestement Mozartian , ecrit considérablement ultérieur concurrently Mozart 's sortie . 


In [13]:
# Fetching English sentences from BUCC data
en_sents_only = [en_sent for en_sent in english_sents['Text']]
fr_sents_only = [fr_sent for fr_sent in french_sents['Text']]

# en_sents_nostp = [[word for word in nltk.word_tokenize(sent)] for sent in en_sents_only]
# fr_vocab = [entry[1] for entry in en_fr_dict_list]

In [14]:
# Fetching gold-standard IDs for each language
en_gold_ids = [id_ for id_ in gold_pairs['en_id']]
fr_gold_ids = [id_ for id_ in gold_pairs['fr_id']]

In [15]:
# For cleaning the gold IDs
def cleanID(id_list):
    for i in range(len(id_list)):
        id_list[i] = re.sub('[^0-9]', '', id_list[i])
        id_list[i] = int(id_list[i].lstrip('0'))
    return id_list

In [16]:
en_ids = cleanID(en_gold_ids)
fr_ids = cleanID(fr_gold_ids)

In [17]:
# Arranging the gold IDs into tuples
# Format: gold_tuples[i] = (en_ids[i], fr_ids[i])

gold_tuples = []
for i in range(len(en_ids)):
    gold_tuples.append((en_ids[i], fr_ids[i]))

In [28]:
start = time.time()
fr_embeddings = embed(fr_sents_only)
end = time.time()
print("Time taken to embed {:} sentences: {:.2f} seconds".format(len(fr_sents_only), (end-start)))

Time taken to embed 271874 sentences: 275.17 seconds


In [29]:
print("Sentence tokenization+embedding rate with SBERT/LaBSE (French): Approx. {:.0f} sents/second".format(len(fr_sents_only)/(end-start)))
#print("N.B. This rate includes the time it takes to take the embeddings off the GPU; the real tokenize/embed rate is faster")

Sentence tokenization+embedding rate with SBERT/LaBSE (French): Approx. 988 sents/second


In [58]:
# Embed the original English sentences . . . 
start = time.time()
en_embeddings = embed(en_sents_only)
end = time.time()
print("Time taken to embed {:} sentences: {:.2f} seconds".format(len(en_sents_only), (end-start)))

Time taken to embed 369810 sentences: 357.61 seconds


In [25]:
# OR embed the translated English sentences
start = time.time()
transl_en_embeddings = embed(transl_en_sents)
end = time.time()
print("Time taken to embed {:} sentences: {:.2f} seconds".format(len(en_sents_only), (end-start)))

Time taken to embed 369810 sentences: 409.21 seconds


In [26]:
print("Sentence tokenization+embedding rate with SBERT/LaBSE (English): Approx. {:.0f} sents/second".format(len(transl_en_sents)/(end-start)))

Sentence tokenization+embedding rate with SBERT/LaBSE (English): Approx. 904 sents/second


In [ ]:
# Embedding English sentences using XLM-RoBERTa
start = time.time()
en_word_embeddings = [[word_embed(word_tokenize(word)) for word in sent.split()] for sent in en_sents_only]
# tok_en_sents = [word_tokenize(sent) for sent in en_sents_only]
# en_word_embeddings = [word_embed(tok_sent) for tok_sent in tok_en_sents]
end = time.time()
print("Time taken to tokenize/embed {:} sentences with XLM-RoBERTa: {:.2f} seconds".format(len(en_sents_only), (end-start)))

In [ ]:
# Embedding French vocab words using XLM-RoBERTa
start = time.time()
fr_vocab_embeddings = [word_embed(word_tokenize(word)) for word in fr_vocab]
end = time.time()
print("Time taken to tokenize/embed {:} words with XLM-RoBERTa: {:.2f} seconds".format(len(fr_vocab), (end-start)))

In [ ]:
####################################################################################
#
# Finding the nearest neighbor for each (translated) English sentence using margin-
# based similarity search (https://www.aclweb.org/anthology/P19-1309.pdf)
#
# Courtesy of https://github.com/facebookresearch/LASER/blob/master/source/
# mine_bitexts.py
#
# Inspiration for any derivative code found below is to be attributed to Mikel 
# Artetxe, Holger Schwenk, and Facebook AI, unless otherwise stated
#
####################################################################################

In [5]:
# Setting up the GPU with FAISS
GPU = faiss.StandardGpuResources()

In [ ]:
# sent1 = 'This is a sentence'
# sent2 = 'C\'est une phrase'
# sent3 = 'Le chien a mangé la tarte'
# sent1_emb = embed(tokenize_sentence(sent1))
# sent2_emb = embed(tokenize_sentence(sent2))
# sent3_emb = embed(tokenize_sentence(sent3))
# print(getCosineSimilarity(sent1_emb, sent2_emb))
# print(getCosineSimilarity(sent1_emb, sent3_emb))
# print('---------')
# sent1_emb = normalize(np.array(sent1_emb.numpy()[0]))
# sent2_emb = normalize(np.array(sent2_emb.numpy()[0]))
# sent3_emb = normalize(np.array(sent3_emb.numpy()[0]))
# print(sent1_emb.dot(sent2_emb))
# print(sent1_emb.dot(sent3_emb))

In [ ]:
# # Converting French and translated English embeddings into matrices

# # print(fr_embeddings[0].numpy()[0])

# fr_emb_mat = np.zeros((len(fr_embeddings), fr_embeddings[0].shape[0]), dtype=np.float32)
# fr_vecs_only = [emb for emb in fr_embeddings]
# for i in range(fr_emb_mat.shape[0]):
#     fr_emb_mat[i] = np.array(fr_vecs_only[i])
    
# en_emb_mat = np.zeros((len(en_embeddings), en_embeddings[0].shape[0]), dtype=np.float32)
# en_vecs_only = [emb for emb in en_embeddings]
# for i in range(en_emb_mat.shape[0]):
#     en_emb_mat[i] = np.array(en_vecs_only[i])

In [38]:
# Normalizing all the embeddings

def normalize(v):
    norm = np.linalg.norm(v)
    if norm != 0:
        return v/norm
    else:
        return v

# for i in range(fr_emb_mat.shape[0]):
#     fr_emb_mat[i] = normalize(fr_emb_mat[i])
# for i in range(en_emb_mat.shape[0]):
#     en_emb_mat[i] = normalize(en_emb_mat[i])

In [6]:
NUM_NEIGHBORS = 4 # Can be adjusted to give better margin score results
BATCH_SIZE = 10000

def knnSearch(src_emb, tgt_emb, k=1, batch_size=1):
    emb_dim = src_emb.shape[1] # Embedding dimension
    num_src_sents = src_emb.shape[0]
    num_tgt_sents = tgt_emb.shape[0]
    cos_sims = np.zeros((num_src_sents, k), dtype=np.float32)
    inds = np.zeros((num_src_sents, k), dtype=np.int64)
    for s_min in range(0, num_src_sents, batch_size):
        #if s_min % 1e2 == 0 and s_min > 0:
            #print("Found {} out of {} forward/backward inds".format(s_min, src_emb.shape[0]))
        s_max = min(s_min + batch_size, num_src_sents)
        src_sims = []
        src_inds = []
        for t_min in range(0, num_tgt_sents, batch_size):
            t_max = min(t_min + batch_size, num_tgt_sents)
            idx = faiss.IndexFlatIP(emb_dim)
            idx = faiss.index_cpu_to_gpu(GPU, 0, idx)
            idx.add(tgt_emb[t_min : t_max])
            src_sim, src_ind = idx.search(src_emb[s_min : s_max], min(k, t_max-t_min))
            src_sims.append(src_sim)
            src_inds.append(src_ind + t_min)
            del idx
        src_sims = np.concatenate(src_sims, axis=1)
        src_inds = np.concatenate(src_inds, axis=1)
        sorted_inds = np.argsort(-src_sims, axis=1)
        for i in range(s_min, s_max):
            for j in range(k):
                cos_sims[i, j] = src_sims[i-s_min, sorted_inds[i-s_min, j]]
                inds[i, j] = src_inds[i-s_min, sorted_inds[i-s_min, j]]
    return cos_sims, inds

In [11]:
# start = time.time()
# knnSearch(transl_en_emb_mat[:1], fr_emb_mat[:1], k=1, batch_size=1)
# end = time.time()
# print(end-start)

# start = time.time()
# transl_en_emb_mat[0].dot(fr_emb_mat[0])
# end = time.time()
# print(end-start)

In [7]:
def directedMeansAndInds(src_emb, tgt_emb, forward=False, backward=False, k=1, batch_size=1):
    assert forward != backward, "Please choose either forward or backward"
    if forward:
        cos_sims, inds = knnSearch(src_emb, tgt_emb, min(tgt_emb.shape[0], k), batch_size)
        return cos_sims.mean(axis=1), inds
    elif backward:
        cos_sims, inds = knnSearch(tgt_emb, src_emb, min(src_emb.shape[0], k), batch_size)
        return cos_sims.mean(axis=1), inds

In [8]:
def margin(a, b, ratio=False, distance=False):
    if ratio:
        return a/b
    elif distance:
        return a-b
    else:
        return a

In [9]:
def scorePair(src_embedding, tgt_embedding, fwd_mean, bwd_mean, r=False, d=False):
    if r:
        return margin(src_embedding.dot(tgt_embedding), (fwd_mean + bwd_mean) / 2, ratio=True)
    elif d:
        return margin(src_embedding.dot(tgt_embedding), (fwd_mean + bwd_mean) / 2, distance=True)
    else:
        return margin(src_embedding.dot(tgt_embedding), _)

In [10]:
def scoreCandidates(src_emb, tgt_emb, candidate_inds, fwd_mean, bwd_mean, r=False, d=False):
    scores = np.zeros(candidate_inds.shape)
    for i in range(scores.shape[0]):
        for j in range(scores.shape[1]):
            k = candidate_inds[i,j]
            scores[i,j] = scorePair(src_emb[i], tgt_emb[k], fwd_mean[i], bwd_mean[k], r, d)
    max_scores = np.zeros((src_emb.shape[0], 3))
    for i in range(scores.shape[0]):
        argmax = np.argmax(scores[i])
        max_ind = candidate_inds[i][argmax]
        max_scores[i] = ((i+1, max_ind+1, np.max(scores[i])))
    return max_scores  
#     max_scores = np.zeros((src_emb.shape[0], 3))
#     for i in range(src_emb.shape[0]):
#         j_max = 0
#         for j in range(tgt_emb.shape[0]):
#             score = scorePair(src_emb[i], tgt_emb[j], fwd_mean[i], bwd_mean[j], r=r, d=d)
#             if score > scorePair(src_emb[i], tgt_emb[j_max], fwd_mean[i], bwd_mean[j_max], r=r, d=d):
#                 j_max = j
#         max_scores[i] = ((i+1, j_max+1, scorePair(src_emb[i], tgt_emb[j_max], fwd_mean[i], bwd_mean[j_max], r=r, d=d)))
# #                 scores[i,j] = scorePair(src_emb[i], tgt_emb[j], fwd_mean[i], bwd_mean[j], r=True)
    
#     # Returns a list of tuples (x_i, y_j) such that the margin score between x_i and y_j is maximized
#     return max_scores

In [11]:
def getMatches(src_emb, 
               tgt_emb,  
               k=1, 
               batch_size=1, 
               ratio=False, 
               distance=False):
    
    fwd_means, fwd_inds = directedMeansAndInds(src_emb, tgt_emb, forward=True, k=k, batch_size=batch_size)
    bwd_means, bwd_inds = directedMeansAndInds(src_emb, tgt_emb, backward=True, k=k, batch_size=batch_size)
    fwd_matches = scoreCandidates(src_emb, tgt_emb, fwd_inds, fwd_means, bwd_means, r=ratio, d=distance)
    bwd_matches = scoreCandidates(tgt_emb, src_emb, bwd_inds, bwd_means, fwd_means, r=ratio, d=distance)
    fwd_matches = [tuple(fwd_matches[i]) for i in range(fwd_matches.shape[0])]
    bwd_matches = [tuple(bwd_matches[i]) for i in range(bwd_matches.shape[0])]
#     matches = []
#     for i in range(margin_scores.shape[0]):
#         matches.append((i+1, np.argmax(margin_scores[i])+1, np.max(margin_scores[i]))) # Add 1 to the indices because the gold training pairs are 1-indexed
    return fwd_matches, bwd_matches

In [12]:
def computeF1(pred_tuples, gold_tuples):
    tp = 0
    fp = 0
    prec = 0
    rec = 0
    f1 = 0
    epsilon = 1e-8 # To prevent division by zero
    for pair in pred_tuples:
        if pair in gold_tuples:
            tp += 1
        else:
            fp += 1 
    prec = tp / (len(pred_tuples) + epsilon)
    rec = tp / len(gold_tuples)
    f1 = 2*prec*rec / (prec+rec+epsilon)
    return f1, prec, rec

In [113]:
for arr in fr_inds:
    for ind in arr:
        ind_sims, ind_inds = knnSearch(en_embeddings[ind:ind+1], fr_embeddings, k=4, batch_size=1000)
        ind_mean = ind_sims.mean(axis=1)
        score = (sample_fr_emb.dot(en_embeddings[ind])) / np.average((ind_mean, fr_sims.mean(axis=1)))
        print(score, ind+1, 88157)

0.29881838
0.45290685 245347 88157
0.35613424
0.5621042 302350 88157
0.2507164
0.39928344 282778 88157
0.3007291
0.4721996 284240 88157


In [59]:
fwd_means, fwd_inds = directedMeansAndInds(en_embeddings, fr_embeddings, forward=True, k=4, batch_size=1000)

In [60]:
bwd_means, bwd_inds = directedMeansAndInds(en_embeddings, fr_embeddings, backward=True, k=4, batch_size=1000)

In [61]:
fwd_margin_scores = np.zeros(fwd_inds.shape)
for i in range(fwd_inds.shape[0]):
    for j in range(fwd_inds.shape[1]):
        tgt_ind = fwd_inds[i,j]
        margin_score = (en_embeddings[i].dot(fr_embeddings[tgt_ind])) / np.average((fwd_means[i], bwd_means[tgt_ind]))
        fwd_margin_scores[i,j] = margin_score
best = np.zeros((fwd_inds.shape[0], 3))
best_inds = fwd_inds[np.arange(en_embeddings.shape[0]), fwd_margin_scores.argmax(axis=1)]
for i in range(fwd_inds.shape[0]):
    best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
    best[i] = ((i+1, best_inds[i]+1, best_score))

In [62]:
bwd_margin_scores = np.zeros(bwd_inds.shape)
for i in range(bwd_inds.shape[0]):
    for j in range(bwd_inds.shape[1]):
        tgt_ind = bwd_inds[i,j]
        margin_score = (fr_embeddings[i].dot(en_embeddings[tgt_ind])) / np.average((bwd_means[i], fwd_means[tgt_ind]))
        bwd_margin_scores[i,j] = margin_score
bwd_best = np.zeros((bwd_inds.shape[0], 3))
best_inds = bwd_inds[np.arange(fr_embeddings.shape[0]), bwd_margin_scores.argmax(axis=1)]
for i in range(bwd_inds.shape[0]):
    best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
    #best_ind = bwd_inds[i][ind]
    bwd_best[i] = ((best_inds[i]+1, i+1, best_score))

In [63]:
fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

pairs_and_scores = []
# pairs_and_scores = list(set(fwd_best) & set(bwd_best))
for fwd_triplet, bwd_triplet in zip(fwd_best, bwd_best):
    pairs_and_scores.append(fwd_triplet)
    pairs_and_scores.append(bwd_triplet)

pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
# #pairs_and_scores = list(set(pairs_and_scores))
concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
#concat_pairs = list(dict.fromkeys(concat_pairs))
#concat_pairs = list(set(concat_pairs))
concat_pairs_int = []
for tup in concat_pairs:
    concat_pairs_int.append((int(tup[0]), int(tup[1])))
concat_pairs = concat_pairs_int
f1, prec, rec = computeF1(concat_pairs, gold_tuples)

margin_scores = [triplet[2] for triplet in pairs_and_scores]

pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv('Data/sent_pairs_fwd_NEW.csv', index=False)
pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv('Data/sent_pairs_bwd_NEW.csv', index=False)
pd.DataFrame({'concat_pairs': concat_pairs}).to_csv('Data/concat_pairs_NEW.csv', index=False)
pd.DataFrame({'margin_scores': margin_scores}).to_csv('Data/margin_scores_NEW.csv', index=False)

print("F1: {:.4f}".format(f1), "Precision: {:.4f}".format(prec), "Recall: {:.4f}".format(rec))

F1: 0.0398 Precision: 0.0203 Recall: 0.9941


In [37]:
# # USING TRANSLATED ENGLISH SENTENCES FOR MINING INSTEAD

# fwd_means, fwd_inds = directedMeansAndInds(transl_en_embeddings, fr_embeddings, forward=True, k=4, batch_size=1000)
# bwd_means, bwd_inds = directedMeansAndInds(transl_en_embeddings, fr_embeddings, backward=True, k=4, batch_size=1000)

# fwd_margin_scores = np.zeros(fwd_inds.shape)
# for i in range(fwd_inds.shape[0]):
#     for j in range(fwd_inds.shape[1]):
#         tgt_ind = fwd_inds[i,j]
#         margin_score = (transl_en_embeddings[i].dot(fr_embeddings[tgt_ind])) / np.average((fwd_means[i], bwd_means[tgt_ind]))
#         fwd_margin_scores[i,j] = margin_score
# best = np.zeros((fwd_inds.shape[0], 3))
# best_inds = fwd_inds[np.arange(transl_en_embeddings.shape[0]), fwd_margin_scores.argmax(axis=1)]
# for i in range(fwd_inds.shape[0]):
#     best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
#     best[i] = ((i+1, best_inds[i]+1, best_score))
    
# bwd_margin_scores = np.zeros(bwd_inds.shape)
# for i in range(bwd_inds.shape[0]):
#     for j in range(bwd_inds.shape[1]):
#         tgt_ind = bwd_inds[i,j]
#         margin_score = (fr_embeddings[i].dot(transl_en_embeddings[tgt_ind])) / np.average((bwd_means[i], fwd_means[tgt_ind]))
#         bwd_margin_scores[i,j] = margin_score
# bwd_best = np.zeros((bwd_inds.shape[0], 3))
# best_inds = bwd_inds[np.arange(fr_embeddings.shape[0]), bwd_margin_scores.argmax(axis=1)]
# for i in range(bwd_inds.shape[0]):
#     best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
#     #best_ind = bwd_inds[i][ind]
#     bwd_best[i] = ((best_inds[i]+1, i+1, best_score))
    
# fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
# bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

# pairs_and_scores = []
# # pairs_and_scores = list(set(fwd_best) & set(bwd_best))
# for fwd_triplet, bwd_triplet in zip(fwd_best, bwd_best):
#     pairs_and_scores.append(fwd_triplet)
#     pairs_and_scores.append(bwd_triplet)

# pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
# # #pairs_and_scores = list(set(pairs_and_scores))
# concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
# #concat_pairs = list(dict.fromkeys(concat_pairs))
# #concat_pairs = list(set(concat_pairs))
# concat_pairs_int = []
# for tup in concat_pairs:
#     concat_pairs_int.append((int(tup[0]), int(tup[1])))
# concat_pairs = concat_pairs_int
# f1, prec, rec = computeF1(concat_pairs, gold_tuples)

# margin_scores = [triplet[2] for triplet in pairs_and_scores]

# pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv('Data/sent_pairs_fwd_T.csv', index=False)
# pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv('Data/sent_pairs_bwd_T.csv', index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv('Data/concat_pairs_T.csv', index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv('Data/margin_scores_T.csv', index=False)

# print("F1: {:.4f}".format(f1), "Precision: {:.4f}".format(prec), "Recall: {:.4f}".format(rec))

F1: 0.0396 Precision: 0.0202 Recall: 0.9938


NameError: name 'all_sent_pairs_NEW' is not defined

In [73]:
# Generating word-by-word translations of English sentences into French using knnSearch

def translateWordByWord(src_sents,
                        src_embs,
                        dict_) -> list:
    
    emb_dim = src_embs[0][0].shape[1]
    wbw_transl = []
    for i in range(len(src_embs)):
        sent_emb_mat = np.zeros((len(src_embs[i]), emb_dim), dtype=np.float32)
        transl = ""
        for j in range(sent_emb_mat.shape[0]):
            sent_emb_mat[j] = np.array(src_embs[i][j])
            word = src_sents[i].split()[j]
            if word in dict_:
                if len(dict_[word])==1:
                    transl += dict_[word][0] + " "
                else:
                    candidates = np.zeros((len(dict_[word]), emb_dim), dtype=np.float32)
                    for k in range(candidates.shape[0]):
                        candidates[k] = word_embed(word_tokenize(dict_[word][k]))
                    _, nn_ind = knnSearch(sent_emb_mat[j:j+1], candidates)
                    transl += dict_[word][int(nn_ind)] + " "
            else:
                transl += word + " "
            if i % 1000 == 0:
                print("Translated sentence {:} of {:}".format(i, len(src_sents)))
        wbw_transl.append(transl)
    return wbw_transl

In [ ]:
start = time.time()
transl_en_sents_wbw = translateWordByWord(en_sents_only, en_word_embeddings, en_fr_dict_dict)
end = time.time()
print(end-start)

In [ ]:
with open('Data/translated_english_sentences_wbw.txt', 'w') as f:
    for sent in transl_en_sents_wbw:
        f.write("%s\n" % sent)

In [ ]:
with open('Data/translated_english_sentences_wbw.txt', 'r') as f:
    transl_en_sents_wbw = f.read().splitlines()

In [31]:
# # Embedding wbw-translated English sentences
start = time.time()
wbw_transl_embeddings = embed(transl_en_sents_wbw)
end = time.time()
print("Time taken to embed {:} sentences: {:.2f} seconds".format(len(transl_en_sents_wbw), end-start))

Time taken to embed 369810 sentences: 366.30 seconds


In [41]:
# Mining sentence pairs and computing F1 score using "code-switched" English sentences

fwd_means, fwd_inds = directedMeansAndInds(wbw_transl_embeddings, fr_embeddings, forward=True, k=4, batch_size=1000)
bwd_means, bwd_inds = directedMeansAndInds(wbw_transl_embeddings, fr_embeddings, backward=True, k=4, batch_size=1000)

fwd_margin_scores = np.zeros(fwd_inds.shape)
for i in range(fwd_inds.shape[0]):
    for j in range(fwd_inds.shape[1]):
        tgt_ind = fwd_inds[i,j]
        margin_score = (wbw_transl_embeddings[i].dot(fr_embeddings[tgt_ind])) / np.average((fwd_means[i], bwd_means[tgt_ind]))
        fwd_margin_scores[i,j] = margin_score
best = np.zeros((fwd_inds.shape[0], 3))
best_inds = fwd_inds[np.arange(wbw_transl_embeddings.shape[0]), fwd_margin_scores.argmax(axis=1)]
for i in range(fwd_inds.shape[0]):
    best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
    best[i] = ((i+1, best_inds[i]+1, best_score))
    
bwd_margin_scores = np.zeros(bwd_inds.shape)
for i in range(bwd_inds.shape[0]):
    for j in range(bwd_inds.shape[1]):
        tgt_ind = bwd_inds[i,j]
        margin_score = (fr_embeddings[i].dot(wbw_transl_embeddings[tgt_ind])) / np.average((bwd_means[i], fwd_means[tgt_ind]))
        bwd_margin_scores[i,j] = margin_score
bwd_best = np.zeros((bwd_inds.shape[0], 3))
best_inds = bwd_inds[np.arange(fr_embeddings.shape[0]), bwd_margin_scores.argmax(axis=1)]
for i in range(bwd_inds.shape[0]):
    best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
    #best_ind = bwd_inds[i][ind]
    bwd_best[i] = ((best_inds[i]+1, i+1, best_score))
    
fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

pairs_and_scores = []
# pairs_and_scores = list(set(fwd_best) & set(bwd_best))
for fwd_triplet, bwd_triplet in zip(fwd_best, bwd_best):
    pairs_and_scores.append(fwd_triplet)
    pairs_and_scores.append(bwd_triplet)

pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
# #pairs_and_scores = list(set(pairs_and_scores))
concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
#concat_pairs = list(dict.fromkeys(concat_pairs))
#concat_pairs = list(set(concat_pairs))
concat_pairs_int = []
for tup in concat_pairs:
    concat_pairs_int.append((int(tup[0]), int(tup[1])))
concat_pairs = concat_pairs_int
f1, prec, rec = computeF1(concat_pairs, gold_tuples)

margin_scores = [triplet[2] for triplet in pairs_and_scores]

pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv('Data/sent_pairs_fwd_wbw.csv', index=False)
pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv('Data/sent_pairs_bwd_wbw.csv', index=False)
pd.DataFrame({'concat_pairs': concat_pairs}).to_csv('Data/concat_pairs_wbw.csv', index=False)
pd.DataFrame({'margin_scores': margin_scores}).to_csv('Data/margin_scores_wbw.csv', index=False)

print("F1: {:.4f}".format(f1), "Precision: {:.4f}".format(prec), "Recall: {:.4f}".format(rec))

F1: 0.0396 Precision: 0.0202 Recall: 0.9924


In [43]:
all_margin_scores_wbw = pd.read_csv('Data/margin_scores_wbw.csv')
all_margin_scores_wbw = [float(score) for score in all_margin_scores_wbw['margin_scores']]
all_sent_pairs_wbw = pd.read_csv('Data/concat_pairs_wbw.csv')
all_sent_pairs_wbw = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs_wbw['concat_pairs']]
all_sent_pairs_wbw = [pair[0] for pair in all_sent_pairs_wbw]

In [ ]:
# print(computeF1(concat_pairs, gold_tuples))

In [ ]:
# start = time.time()
# test_pairs_fwd = getMatches(x, y, k=NUM_NEIGHBORS, batch_size=BATCH_SIZE, ratio=True)
# test_pairs_bwd = getMatches(y, x, k=NUM_NEIGHBORS, batch_size=BATCH_SIZE, ratio=True)
# concat_pairs = []
# for fwd_triplet, bwd_triplet in zip(test_pairs_fwd, test_pairs_bwd):
#     concat_pairs.append(fwd_triplet[0:2])
#     concat_pairs.append((bwd_triplet[1], bwd_triplet[0]))
# concat_pairs = list(set(concat_pairs))
# f1, prec, rec = computeF1(concat_pairs, gold_tuples)
# end = time.time()
# print(end-start)

In [ ]:
# # # Getting matches for real now

# start = time.time()

# sent_pairs_fwd, sent_pairs_bwd = getMatches(en_emb_mat, fr_emb_mat, k=NUM_NEIGHBORS, batch_size=BATCH_SIZE, ratio=True)
# pairs_and_scores = []
# for fwd_triplet, bwd_triplet in zip(sent_pairs_fwd, sent_pairs_bwd):
#     pairs_and_scores.append(fwd_triplet)
#     pairs_and_scores.append((bwd_triplet[1], bwd_triplet[0], bwd_triplet[2]))
#     #concat_pairs.append(fwd_triplet[0:2])
#     #margin_scores.append(fwd_triplet[2])
#     #concat_pairs.append((bwd_triplet[1], bwd_triplet[0]))
#     #margin_scores.append(bwd_triplet[2])

# pairs_and_scores = list(set(pairs_and_scores))
# concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
# concat_pairs = list(set(concat_pairs))
# #concat_pairs = list(set(concat_pairs))
# concat_pairs_int = []
# for tup in concat_pairs:
#     concat_pairs_int.append((int(tup[0]), int(tup[1])))
# concat_pairs = concat_pairs_int
# f1, prec, rec = computeF1(concat_pairs, gold_tuples)

# margin_scores = [triplet[2] for triplet in pairs_and_scores]

# pd.DataFrame({'sent_pairs_fwd': sent_pairs_fwd}).to_csv('Data/sent_pairs_fwd_NEW.csv', index=False)
# pd.DataFrame({'sent_pairs_bwd': sent_pairs_bwd}).to_csv('Data/sent_pairs_bwd_NEW.csv', index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv('Data/concat_pairs_NEW.csv', index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv('Data/margin_scores_NEW.csv', index=False)

# print("F1: {:.4f}".format(f1), "Precision: {:.4f}".format(prec), "Recall: {:.4f}".format(rec))

# end = time.time()
# print("Time taken to find pairs: {:.2f} seconds".format((end-start)))

In [57]:
print(len(margin_scores))
print(len(all_sent_pairs))

445050
445050


In [10]:
all_margin_scores = pd.read_csv('Data/margin_scores.csv')
all_margin_scores_NEW = pd.read_csv('Data/margin_scores_NEW.csv')

In [11]:
all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
all_margin_scores_NEW = [float(score) for score in all_margin_scores_NEW['margin_scores']]

In [12]:
# Reading the sentence pairs back in
all_sent_pairs = pd.read_csv('Data/concat_pairs.csv')
all_sent_pairs_NEW = pd.read_csv('Data/concat_pairs_NEW.csv')

In [13]:
# Converting strings back into tuples
all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
all_sent_pairs_NEW = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs_NEW['concat_pairs']]

In [14]:
# Extracting just a list of tuples
all_sent_pairs = [pair[0] for pair in all_sent_pairs]
all_sent_pairs_NEW = [pair[0] for pair in all_sent_pairs_NEW]

In [15]:
test_sim, test_ind =  knnSearch(fr_embeddings[237295:237296], en_embeddings, k=1, batch_size=1000)
ts, ti = knnSearch(en_embeddings[100352:100353], fr_embeddings, k=1, batch_size=1000)

NameError: name 'knnSearch' is not defined

In [ ]:
print(test_ind, test_sim)
print(ti, ts)

In [57]:
print(all_margin_scores_NEW[:10])

[0.5242267847061157, 0.5467867255210876, 0.5596277713775635, 0.5174039602279663, 0.5158339142799377, 0.5208346843719482, 0.6611602306365967, 0.5113794803619385, 0.4711534380912781, 0.5581844449043274]


In [48]:
print(en_embeddings[142313].dot(fr_embeddings[260047]))

0.5495959


In [24]:
# # Checking to see that the pairs we just read in match those we wrote out
# print(computeF1(all_sent_pairs, gold_tuples))
# print("Should print '(0.039763418495595414, 0.02028760813391754, 0.9937266123706802)'")

(0.039763418495595414, 0.02028760813391754, 0.9937266123706802)
Should print '(0.039763418495595414, 0.02028760813391754, 0.9937266123706802)'


In [25]:
#######################################################################################
# 
# Now, we try to optimize the F1 score using other features
# of the sentence pairs we already have.
#
# Benchmark to beat: 92.5 (https://arxiv.org/pdf/2004.09813.pdf).
#
# After the F1 score has been optimized on Fr-En using these features, we can
# test how these same features transfer to other languages in the BUCC training
# sets, and then to low-resource languages not in the BUCC data (e.g. Kazakh, Gujarati,
# Somali)
#
#######################################################################################

In [20]:
print(len(all_margin_scores)==len(all_sent_pairs)) # Should print True

True


In [155]:
print(max(all_margin_scores), max(all_margin_scores_NEW))
print(min(all_margin_scores), min(all_margin_scores_NEW))
print(np.average(all_margin_scores), np.average(all_margin_scores_NEW))
print(all_margin_scores==all_margin_scores_NEW)
_ = [(sim1, sim2) for sim1, sim2 in zip(all_margin_scores, all_margin_scores_NEW) if sim1==sim2]

1.605146272722705 1.0
0.7669204422819055 0.29242923855781555
1.0070340326908442 0.5864986017554328
False


In [195]:
print(np.average(all_cosine_similarities))
print(np.min(all_cosine_similarities))

0.6222602317800019
-0.05660247802734375


In [59]:
# Tuning the margin score threshold to be the max value t such that recall > 0.99

MARGIN_THRESHOLD = 1.17
CS_THRESHOLD = 0.84
updated_sent_pairs = [] # Where we'll keep the new sentence pairs filtered from the initial set

# for i in range(len(all_sent_pairs)):
#     if all_cosine_similarities[i] > CS_THRESHOLD:
#         updated_sent_pairs.append(all_sent_pairs[i])

# print(computeF1(updated_sent_pairs, gold_tuples))

updated_sent_pairs_wbw = []

for i in range(len(all_sent_pairs_wbw)):
    if all_margin_scores_wbw[i] > MARGIN_THRESHOLD:
        updated_sent_pairs_wbw.append(all_sent_pairs_wbw[i])
        
print(computeF1(updated_sent_pairs_wbw, gold_tuples))

(0.9146169632529533, 0.9006615450267811, 0.9290116662998019)


In [74]:
fn_en = [en_sents_only[pair[0]-1] for pair in gold_tuples if pair not in updated_sent_pairs_NEW]
fn_fr = [fr_sents_only[pair[1]-1] for pair in gold_tuples if pair not in updated_sent_pairs_NEW]
fp_en = [en_sents_only[pair[0]-1] for pair in updated_sent_pairs_NEW if pair not in gold_tuples]
fp_fr = [fr_sents_only[pair[1]-1] for pair in updated_sent_pairs_NEW if pair not in gold_tuples]

In [78]:
# pd.DataFrame({"en": fn_en, "fr": fn_fr}).to_csv('Data/false_negatives.csv', index=False)
# pd.DataFrame({"en": fp_en, "fr": fp_fr}).to_csv('Data/false_positives.csv', index=False)

In [25]:
false_negatives = pd.read_csv('Data/false_negatives.csv')
false_positives = pd.read_csv('Data/false_positives.csv')

In [28]:
# Recomputing F1-score by estimating number of incorrectly-flagged false positives and
# false negatives

f1, prec, rec = computeF1(updated_sent_pairs_NEW, gold_tuples)

NUM_TPS = int(len(updated_sent_pairs_NEW)*prec)
NUM_FNS = len(gold_tuples) - int(len(gold_tuples)*rec)

print(len(updated_sent_pairs_NEW))

# Ratio of "false" false positive sentence pairs to "true" false positive sentence pairs
# i.e. 72 out of the first 100 sentence pairs flagged as false positives were deemed to be
# valid translations of one another
FFP_RATIO = 0.72

# Ratio of "false" false negative sentences pairs to "true" false negatives,
# computed as above
FFN_RATIO = 0

NUM_FALSE_FPS = 0.72*len(false_positives)
NUM_FALSE_FNS = 0
UPDATED_PREC = (NUM_TPS + NUM_FALSE_FPS) / len(updated_sent_pairs_NEW)
UPDATED_REC = 0
UPDATED_F1 = 2*UPDATED_PREC*rec / (UPDATED_PREC + rec)
print(UPDATED_PREC, UPDATED_F1)

9419
0.9740312135046183 0.9571780057002534


In [29]:
false_positive_ids = [pair for pair in updated_sent_pairs_NEW if pair not in gold_tuples]

In [30]:
gold_tuples_new = [tup for tup in gold_tuples]
for id_ in false_positive_ids:
    k = random.uniform(0, 1)
    if k <= FFP_RATIO:
        gold_tuples_new.append(id_)

In [31]:
print(len(gold_tuples_new)-len(gold_tuples)) # Number of false false positives in training data

638


In [32]:
print(computeF1(updated_sent_pairs_NEW, gold_tuples_new))

(0.9598286529958574, 0.9753689351300824, 0.9447758124228712)


In [25]:
parser = benepar.Parser("benepar_fr")
sample_tree = parser.parse(fr_sents_only[0])
print(sample_tree, fr_sents_only[0])

(SENT
  (VN (CLS-SUJ Il) (V peut))
  (VPinf-OBJ
    (VN (VINF être) (VPP divisé))
    (PP-P_OBJ (P en) (NP (DET trois) (NC zones)))
    (PP-MOD
      (P+D+ (P en) (NC fonction) (P+D du))
      (NP (NC relief))
      (COORD (PONCT ,) (PP (P+D du) (NP (NC climat))))
      (COORD (PONCT ,) (PP (P+D du) (NP (NC peuplement))))
      (COORD (CC et) (PP (P de) (NP (NC l'économie))))))
  (PONCT :)) Il peut être divisé en trois zones en fonction du relief, du climat, du peuplement et de l'économie:


In [37]:
# Experimenting with syntax tree operations

print(sample_tree.label()) # Returns the node label of the entire sentence (maximal projection)
print(sample_tree.leaves()) # Returns leaves of tree as a list; should be identical to input sentence
print(sample_tree.flatten()) # Returns Tree.label() + Tree.leaves()
print(sample_tree.height()) # Returns the height of the tree
print(sample_tree.subtrees) # Returns all subtrees within the tree, ordered sequentially
print(sample_tree.productions) # Seems to return the same thing as Tree.subtrees??
print(sample_tree.pos) # Returns pos-tagged sentence

SENT
['Il', 'peut', 'être', 'divisé', 'en', 'trois', 'zones', 'en', 'fonction', 'du', 'relief', ',', 'du', 'climat', ',', 'du', 'peuplement', 'et', 'de', "l'économie", ':']
(SENT
  Il
  peut
  être
  divisé
  en
  trois
  zones
  en
  fonction
  du
  relief
  ,
  du
  climat
  ,
  du
  peuplement
  et
  de
  l'économie
  :)
8
<bound method Tree.subtrees of Tree('SENT', [Tree('VN', [Tree('CLS-SUJ', ['Il']), Tree('V', ['peut'])]), Tree('VPinf-OBJ', [Tree('VN', [Tree('VINF', ['être']), Tree('VPP', ['divisé'])]), Tree('PP-P_OBJ', [Tree('P', ['en']), Tree('NP', [Tree('DET', ['trois']), Tree('NC', ['zones'])])]), Tree('PP-MOD', [Tree('P+D+', [Tree('P', ['en']), Tree('NC', ['fonction']), Tree('P+D', ['du'])]), Tree('NP', [Tree('NC', ['relief'])]), Tree('COORD', [Tree('PONCT', [',']), Tree('PP', [Tree('P+D', ['du']), Tree('NP', [Tree('NC', ['climat'])])])]), Tree('COORD', [Tree('PONCT', [',']), Tree('PP', [Tree('P+D', ['du']), Tree('NP', [Tree('NC', ['peuplement'])])])]), Tree('COORD', [Tree('

In [56]:
# Storing a list of subtree nodes (projections) from the sentence
subtree_nodes = []
for subtree in sample_tree.subtrees():
    subtree_nodes.append(subtree.label())
print(subtree_nodes)

print("----------------------------------------------------------------")

# Storing a list of POS tags from the sentence
pos = []
for node in sample_tree.pos():
    pos.append(node[1])
print(pos)

['SENT', 'VN', 'CLS-SUJ', 'V', 'VPinf-OBJ', 'VN', 'VINF', 'VPP', 'PP-P_OBJ', 'P', 'NP', 'DET', 'NC', 'PP-MOD', 'P+D+', 'P', 'NC', 'P+D', 'NP', 'NC', 'COORD', 'PONCT', 'PP', 'P+D', 'NP', 'NC', 'COORD', 'PONCT', 'PP', 'P+D', 'NP', 'NC', 'COORD', 'CC', 'PP', 'P', 'NP', 'NC', 'PONCT']
----------------------------------------------------------------
['CLS-SUJ', 'V', 'VINF', 'VPP', 'P', 'DET', 'NC', 'P', 'NC', 'P+D', 'NC', 'PONCT', 'P+D', 'NC', 'PONCT', 'P+D', 'NC', 'CC', 'P', 'NC', 'PONCT']


In [75]:
parser = benepar.Parser("benepar_fr")

In [82]:
sample_tree_2 = parser.parse([en2fr(en_word)[0] for en_word in re.findall(r'\w+', en_sents_only[0])])
stree_list = []
for stree in sample_tree_2.subtrees():
    stree_list.append(stree.label())
print(stree_list)

['SENT', 'VN', 'VS', 'NP-SUJ', 'NPP', 'CC', 'NP-SUJ', 'NPP', 'NPP', 'VN', 'V', 'ADV', 'VPP', 'Ssub-OBJ', 'PROWH', 'PP', 'P+D', 'NP', 'PRO', 'ADV', 'ADJ', 'VPP']


In [92]:
print([en2fr(en_word)[0] for en_word in re.findall(r'\w+', en_sents_only[0])])

['plaise', 'Pygmalion', 'ni', 'Frankenstein', 'humanité', 'doit', 'longtemps', 'fasciné', 'Qu', 'aux', 'Aucune', 'beaucoup', 'artificiel', 'sauvé']


In [99]:
print(en_sents_only[0])
print(' '.join([en2fr(word)[0] for word in re.findall(r'\w+', en_sents_only[0])]))

Whether Pygmalion or Frankenstein, humanity has been fascinated with the idea of artificial life.
plaise Pygmalion ni Frankenstein humanité doit longtemps fasciné Qu aux Aucune beaucoup artificiel sauvé


In [87]:
print(pylcs.lcs2(''.join(stree_list), ''.join(subtree_nodes)))

7


In [21]:
# # Using constituent overlap as a filtering mechanism

# parser = benepar.Parser('benepar_fr')
# en2fr = w2w('en', 'fr')

# # Returns length of longest common substring between joined list of constituents
# # for each sentence
# def getConstituentOverlaps(en_sents: list, fr_sents: list) -> list:
#     print("Creating French constituent trees . . .")
#     fr_trees = parser.parse_sents(fr_sents)
#     print("French trees complete!")
#     print("Creating word-by-word translations of English sentences . . .")
#     transl_en_sents = []
#     i = 0
#     for en_sent in en_sents:
#         for en_word in re.findall(r'\w+', en_sent):
#             transl_en_sent = []
#             try:
#                 transl_en_sent.append(en2fr(en_word)[0])
#             except:
#                 transl_en_sent.append(en_word)
#         transl_en_sents.append(transl_en_sent)
#         i += 1
#         if i % 1e5 == 0:
#             print("Translated sentence {:} of {:}".format(i, len(en_sents)), flush=True)
#     print("Translations complete!")
#     print("Creating translated English constituent trees . . .")
#     transl_en_trees = parser.parse_sents([' '.join(sent) for sent in transl_en_sents])
#     print("Translated English trees complete!")
#     print("Parsing French constituents . . .")
#     all_fr_constituents = []
#     j = 0
#     start1 = time.time()
#     for tree in fr_trees:
#         constituents = ''.join([stree.label() for stree in tree.subtrees()])
#         all_fr_constituents.append(constituents)
#         j += 1
#         if j % 1e4 == 0:
#             end1 = time.time()
#             print("Parsed sentence {:} of {:}".format(j, len(fr_sents)), flush=True)
#             print("Time taken: {:.1f} seconds".format(end1-start1))
#             start1 = time.time()
#     print("French constituent parsing complete!")
#     print("Parsing translated English constituents . . .")
#     all_transl_en_constituents = []
#     k = 0
#     start2 = time.time()
#     for tree in transl_en_trees:
#         constituents = ''.join([stree.label() for stree in tree.subtrees()])
#         all_transl_en_constituents.append(constituents)
#         k += 1
#         if k % 1e4 == 0:
#             end2 = time.time()
#             print("Parsed sentence {:} of {:}".format(k, len(en_sents)), flush=True)
#             print("Time taken: {:.1f} seconds".format(end2-start2))
#     print("Translated English constituent parsing complete!")
#     print("Computing longest continuous constituent overlaps . . .")
#     overlap = [pylcs.lcs2(fr_consts, transl_en_consts) for fr_consts, transl_en_consts in zip(all_fr_constituents, all_transl_en_constituents)]
#     print("Constituent overlap computations complete!")
#     return overlap

In [16]:
# all_en_sents = [en_sents_only[pair[0]-1] for pair in all_sent_pairs_NEW]
# all_fr_sents = [fr_sents_only[pair[1]-1] for pair in all_sent_pairs_NEW]

In [ ]:
# start = time.time()
# all_constituent_overlaps = getConstituentOverlaps(all_en_sents, all_fr_sents)
# end = time.time()
# print("Time elapsed: {:.1f} seconds".format(end-start))

Creating French constituent trees . . .
French trees complete!
Creating word-by-word translations of English sentences . . .
Translated sentence 100000 of 445054
Translated sentence 200000 of 445054
Translated sentence 300000 of 445054
Translated sentence 400000 of 445054
Translations complete!
Creating translated English constituent trees . . .
Translated English trees complete!
Parsing French constituents . . .
Parsed sentence 10000 of 445054
Time taken: 146.2 seconds
Parsed sentence 20000 of 445054
Time taken: 140.9 seconds
Parsed sentence 30000 of 445054
Time taken: 139.7 seconds
Parsed sentence 40000 of 445054
Time taken: 141.9 seconds


In [ ]:
# with open('Data/constituent_overlaps.txt', 'w') as f:
#     for overlap in all_constituent_overlaps:
#         f.write(str(overlap)+'\n')

In [33]:
with open('Data/constituent_overlaps.txt', 'r') as f:
    all_constituent_overlaps = f.read().splitlines()
all_constituent_overlaps = [int(val) for val in all_constituent_overlaps]

In [ ]:
# Tuning the margin score threshold to be the max value t such that recall > 0.99

MARGIN_THRESHOLD = 1.19
CONST_THRESHOLD = 7

updated_sent_pairs_NEW = []

for i in range(len(all_sent_pairs_NEW)):
    if all_margin_scores_NEW[i] > MARGIN_THRESHOLD and all_constituent_overlaps[i] > CONST_THRESHOLD:
        updated_sent_pairs_NEW.append(all_sent_pairs_NEW[i])
        
print(computeF1(updated_sent_pairs_NEW, gold_tuples_new))

In [49]:
print(max(all_constituent_overlaps))

11


In [132]:
# Using length ratio as a filtering feature

LENGTH_RATIO = 1.1
updated_sent_pairs = []

for i in range(len(all_sent_pairs_NEW)):
    en_ind, fr_ind = (all_sent_pairs_NEW[i][0]-1, all_sent_pairs_NEW[i][1]-1)
    if all_margin_scores_NEW[i] > MARGIN_THRESHOLD and ( (1-LENGTH_RATIO)*len(en_sents_only[en_ind]) < len(fr_sents_only[fr_ind]) < (1+LENGTH_RATIO)*len(en_sents_only[en_ind]) ):
        updated_sent_pairs.append(all_sent_pairs_NEW[i])

print(computeF1(updated_sent_pairs, gold_tuples))

(0.9239083391429174, 0.9076236992982505, 0.9407880255337883)


In [23]:
# Translating English sentences to French in order to calculate BLEU score
#
#####################################################################################################
# Using the Helsinki-NLP / OPUS NMT model from HuggingFace to translate En BUCC train data to French
#####################################################################################################
#
#################### *** SKIP THIS if sentences have already been translated *** ####################

In [54]:
# First, installing/importing the necessary depenencies . . . 

# !pip install mosestokenizer
from transformers import MarianMTModel, MarianTokenizer

In [55]:
translation_model_name = 'Helsinki-NLP/opus-mt-en-trk'
marian_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)
translation_model.cuda().half() # Puts translation model on GPU

Downloading:   0%|          | 0.00/305M [00:00<?, ?B/s]

MarianMTModel(
  (model): BartModel(
    (shared): Embedding(61674, 512, padding_idx=61673)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(61674, 512, padding_idx=61673)
      (embed_positions): SinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): EncoderLayer

In [136]:
def translateToFrench(sent):
    tokenized = marian_tokenizer.prepare_seq2seq_batch(sent, return_tensors='pt')
    tokenized.to(device)
    translated = translation_model.generate(**tokenized)
    decoded = [marian_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    tokenized.to('cpu')
    return decoded

In [142]:
# transl_sents_test = []
# start = time.time()
# # for sent in en_sents_only[3000:3010]:
# #     transl_sents_test.append(translateToFrench([sent]))
# translated = translateToFrench(en_sents_only[5000:5100])
# end = time.time()
# print(end-start)

3.2649881839752197


In [24]:
en_ids_to_translate = [pair[0] for pair in all_sent_pairs]

In [25]:
en_sents_to_translate = [en_sents_only[id_-1] for id_ in en_ids_to_translate]

In [ ]:
### Translating English sentences to French

NUM_EN_SENTS = len(en_sents_only)
BATCH_SIZE = 100
NUM_BATCHES = (NUM_EN_SENTS // BATCH_SIZE) + 1
i = 0
transl_en_sents = []
start = time.time()
for i in range(NUM_BATCHES):
    transl_en_sents.append(translateToFrench(en_sents_only[i*BATCH_SIZE : (i+1)*BATCH_SIZE]))
    print("Completed batch {:} of {:}".format(i+1, NUM_BATCHES))
end = time.time()
print("Time taken: {:.3f}".format(end-start))

Completed batch 1 of 3699
Completed batch 2 of 3699
Completed batch 3 of 3699
Completed batch 4 of 3699
Completed batch 5 of 3699
Completed batch 6 of 3699
Completed batch 7 of 3699
Completed batch 8 of 3699
Completed batch 9 of 3699
Completed batch 10 of 3699
Completed batch 11 of 3699
Completed batch 12 of 3699
Completed batch 13 of 3699
Completed batch 14 of 3699
Completed batch 15 of 3699
Completed batch 16 of 3699
Completed batch 17 of 3699
Completed batch 18 of 3699
Completed batch 19 of 3699
Completed batch 20 of 3699
Completed batch 21 of 3699
Completed batch 22 of 3699
Completed batch 23 of 3699
Completed batch 24 of 3699
Completed batch 25 of 3699
Completed batch 26 of 3699
Completed batch 27 of 3699
Completed batch 28 of 3699
Completed batch 29 of 3699
Completed batch 30 of 3699
Completed batch 31 of 3699
Completed batch 32 of 3699
Completed batch 33 of 3699
Completed batch 34 of 3699
Completed batch 35 of 3699
Completed batch 36 of 3699
Completed batch 37 of 3699
Completed 

Completed batch 298 of 3699
Completed batch 299 of 3699
Completed batch 300 of 3699
Completed batch 301 of 3699
Completed batch 302 of 3699
Completed batch 303 of 3699
Completed batch 304 of 3699
Completed batch 305 of 3699
Completed batch 306 of 3699
Completed batch 307 of 3699
Completed batch 308 of 3699
Completed batch 309 of 3699
Completed batch 310 of 3699
Completed batch 311 of 3699
Completed batch 312 of 3699
Completed batch 313 of 3699
Completed batch 314 of 3699
Completed batch 315 of 3699
Completed batch 316 of 3699
Completed batch 317 of 3699
Completed batch 318 of 3699
Completed batch 319 of 3699
Completed batch 320 of 3699
Completed batch 321 of 3699
Completed batch 322 of 3699
Completed batch 323 of 3699
Completed batch 324 of 3699
Completed batch 325 of 3699
Completed batch 326 of 3699
Completed batch 327 of 3699
Completed batch 328 of 3699
Completed batch 329 of 3699
Completed batch 330 of 3699
Completed batch 331 of 3699
Completed batch 332 of 3699
Completed batch 333 

Completed batch 591 of 3699
Completed batch 592 of 3699
Completed batch 593 of 3699
Completed batch 594 of 3699
Completed batch 595 of 3699
Completed batch 596 of 3699
Completed batch 597 of 3699
Completed batch 598 of 3699
Completed batch 599 of 3699
Completed batch 600 of 3699
Completed batch 601 of 3699
Completed batch 602 of 3699
Completed batch 603 of 3699
Completed batch 604 of 3699
Completed batch 605 of 3699
Completed batch 606 of 3699
Completed batch 607 of 3699
Completed batch 608 of 3699
Completed batch 609 of 3699
Completed batch 610 of 3699
Completed batch 611 of 3699
Completed batch 612 of 3699
Completed batch 613 of 3699
Completed batch 614 of 3699
Completed batch 615 of 3699
Completed batch 616 of 3699
Completed batch 617 of 3699
Completed batch 618 of 3699
Completed batch 619 of 3699
Completed batch 620 of 3699
Completed batch 621 of 3699
Completed batch 622 of 3699
Completed batch 623 of 3699
Completed batch 624 of 3699
Completed batch 625 of 3699
Completed batch 626 

Completed batch 884 of 3699
Completed batch 885 of 3699
Completed batch 886 of 3699
Completed batch 887 of 3699
Completed batch 888 of 3699
Completed batch 889 of 3699
Completed batch 890 of 3699
Completed batch 891 of 3699
Completed batch 892 of 3699
Completed batch 893 of 3699
Completed batch 894 of 3699
Completed batch 895 of 3699
Completed batch 896 of 3699
Completed batch 897 of 3699
Completed batch 898 of 3699
Completed batch 899 of 3699
Completed batch 900 of 3699
Completed batch 901 of 3699
Completed batch 902 of 3699
Completed batch 903 of 3699
Completed batch 904 of 3699
Completed batch 905 of 3699
Completed batch 906 of 3699
Completed batch 907 of 3699
Completed batch 908 of 3699
Completed batch 909 of 3699
Completed batch 910 of 3699
Completed batch 911 of 3699
Completed batch 912 of 3699
Completed batch 913 of 3699
Completed batch 914 of 3699
Completed batch 915 of 3699
Completed batch 916 of 3699
Completed batch 917 of 3699
Completed batch 918 of 3699
Completed batch 919 

Completed batch 1171 of 3699
Completed batch 1172 of 3699
Completed batch 1173 of 3699
Completed batch 1174 of 3699
Completed batch 1175 of 3699
Completed batch 1176 of 3699
Completed batch 1177 of 3699
Completed batch 1178 of 3699
Completed batch 1179 of 3699
Completed batch 1180 of 3699
Completed batch 1181 of 3699
Completed batch 1182 of 3699
Completed batch 1183 of 3699
Completed batch 1184 of 3699
Completed batch 1185 of 3699
Completed batch 1186 of 3699
Completed batch 1187 of 3699
Completed batch 1188 of 3699
Completed batch 1189 of 3699
Completed batch 1190 of 3699
Completed batch 1191 of 3699
Completed batch 1192 of 3699
Completed batch 1193 of 3699
Completed batch 1194 of 3699
Completed batch 1195 of 3699
Completed batch 1196 of 3699
Completed batch 1197 of 3699
Completed batch 1198 of 3699
Completed batch 1199 of 3699
Completed batch 1200 of 3699
Completed batch 1201 of 3699
Completed batch 1202 of 3699
Completed batch 1203 of 3699
Completed batch 1204 of 3699
Completed batc

Completed batch 1454 of 3699
Completed batch 1455 of 3699
Completed batch 1456 of 3699
Completed batch 1457 of 3699
Completed batch 1458 of 3699
Completed batch 1459 of 3699
Completed batch 1460 of 3699
Completed batch 1461 of 3699
Completed batch 1462 of 3699
Completed batch 1463 of 3699
Completed batch 1464 of 3699
Completed batch 1465 of 3699
Completed batch 1466 of 3699
Completed batch 1467 of 3699
Completed batch 1468 of 3699
Completed batch 1469 of 3699
Completed batch 1470 of 3699
Completed batch 1471 of 3699
Completed batch 1472 of 3699
Completed batch 1473 of 3699
Completed batch 1474 of 3699
Completed batch 1475 of 3699
Completed batch 1476 of 3699
Completed batch 1477 of 3699
Completed batch 1478 of 3699
Completed batch 1479 of 3699
Completed batch 1480 of 3699
Completed batch 1481 of 3699
Completed batch 1482 of 3699
Completed batch 1483 of 3699
Completed batch 1484 of 3699
Completed batch 1485 of 3699
Completed batch 1486 of 3699
Completed batch 1487 of 3699
Completed batc

Completed batch 1737 of 3699
Completed batch 1738 of 3699
Completed batch 1739 of 3699
Completed batch 1740 of 3699
Completed batch 1741 of 3699
Completed batch 1742 of 3699
Completed batch 1743 of 3699
Completed batch 1744 of 3699
Completed batch 1745 of 3699
Completed batch 1746 of 3699
Completed batch 1747 of 3699
Completed batch 1748 of 3699
Completed batch 1749 of 3699
Completed batch 1750 of 3699
Completed batch 1751 of 3699
Completed batch 1752 of 3699
Completed batch 1753 of 3699
Completed batch 1754 of 3699
Completed batch 1755 of 3699
Completed batch 1756 of 3699
Completed batch 1757 of 3699
Completed batch 1758 of 3699
Completed batch 1759 of 3699
Completed batch 1760 of 3699
Completed batch 1761 of 3699
Completed batch 1762 of 3699
Completed batch 1763 of 3699
Completed batch 1764 of 3699
Completed batch 1765 of 3699
Completed batch 1766 of 3699
Completed batch 1767 of 3699
Completed batch 1768 of 3699
Completed batch 1769 of 3699
Completed batch 1770 of 3699
Completed batc

In [ ]:
with open('Data/translated_english_sentences_all.txt', 'w') as f:
    for batch in transl_en_sents:
        for sent in batch:
            f.write("%s\n" % sent)

In [ ]:
with open('Data/translated_english_sentences_all.txt', 'r') as f:
    transl_en_sents = f.read().splitlines()

In [110]:
print(len(transl_en_sents))

445050


In [188]:
# Using BLEU score as a filtering mechanism

smoother = nltk.translate.bleu_score.SmoothingFunction()
#W = 8 # Weights coefficient
#weights = W*[1/W]
start = time.time()
bleu_scores = []
# for i in range(len(all_sent_pairs[:10000])):
#     fr_sent_id = all_sent_pairs[i][1]
#     fr_sent = fr_sents_only[fr_sent_id-1]
#     fr_sent = fr_sent.split()
#     transl_en_sent = transl_en_sents[i].split()
#     score = sentence_bleu(fr_sent, transl_en_sent) 
#     bleu_scores.append(score)
    
bleu_scores = [sentence_bleu([fr_sents_only[all_sent_pairs[i][1]-1].split()], transl_en_sents[i].split(), smoothing_function=smoother.method7) for i in range(len(all_sent_pairs))]
    
# bleu_scores = [sentence_bleu([fr_sents_only[pair[1]-1].split()], transl_en_sents[all_sent_pairs.index(pair)].split()) for pair in all_sent_pairs[:20000]]
end = time.time()
print(end-start)

85.54857420921326


In [298]:
# x = fr_sents_only[all_sent_pairs[129541][1]-1].split()
# y = transl_en_sents[129541].split()
# W = 7
# weights = W*[1/W]
# print(sentence_bleu([x], y, smoothing_function=smoother.method0, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method1, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method2, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method3, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method4, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method5, weights=weights))
# print(sentence_bleu([x], y, smoothing_function=smoother.method7, weights=weights))

In [199]:
# Multiplying BLEU scores by 100

for i in range(len(bleu_scores)):
    bleu_scores[i] *= 100

In [200]:
print(len(bleu_scores))
print(max(bleu_scores))
print(min(bleu_scores))
print(np.average(bleu_scores))

445050
111.67470964180197
0.0
26.9367279381114


In [305]:
BLEU_THRESHOLD = 60

updated_sent_pairs = []
for i in range(len(all_sent_pairs)):
    if all_margin_scores[i] > THRESHOLD and bleu_scores[i] > BLEU_THRESHOLD:
        updated_sent_pairs.append(all_sent_pairs[i])

print(computeF1(updated_sent_pairs, gold_tuples))

(0.31575117937724306, 0.9089005235554508, 0.19106317411402157)


In [24]:
# Testing corpus-level BLEU score of translated English sentences
transl_en_sents_gold = []
fr_sents_gold = []
for tup in gold_tuples:
    if tup in all_sent_pairs:
        transl_en_sents_gold.append(transl_en_sents[all_sent_pairs.index(tup)])
        fr_sents_gold.append(fr_sents_only[tup[1]-1])

In [106]:
print(transl_en_sents_gold[:10])
print(fr_sents_gold[:10])

['Mais en fin de compte, Merkel a accepté un fonds de sauvetage permanent pour la zone euro.', "Mais le changement de régime coercitif n'a jamais été la clé de la transition démocratique.", "Dans d'autres cas, les sorties de capitaux ne font qu'aggraver des situations déjà difficiles.", "Si les humanités sont devenues obsolètes, alors il se peut que l'humanité perde sa salive.", "En conséquence, la politique mondiale n'est plus la seule province de gouvernement.", "Il a utilisé une partie de sa grande richesse pour verser de l'argent dans les zones rurales.", "Une technopole reconnaît également le rôle crucial de la coopération, de la réflexion stratégique et de l'adaptation.", "L'exploitation minière et l'agriculture des métaux ont connu des améliorations similaires dans l'ensemble du pays.", 'La seule façon de sortir de ce dilemme est de fixer un objectif réalisable et réaliste.', 'Il est certain que les réserves sont importantes pour aplanir les déséquilibres dans un régime de taux 

In [25]:
fr_references_gold = [[fr_sent.split()] for fr_sent in fr_sents_gold]
fr_hypotheses_gold = [fr_sent.split() for fr_sent in transl_en_sents_gold]

In [208]:
corp_bleu_gold = nltk.translate.bleu_score.corpus_bleu(fr_references_gold, fr_hypotheses_gold)

In [209]:
sent_bleu_gold = [nltk.translate.bleu_score.sentence_bleu(fr_references_gold[i], fr_hypotheses_gold[i]) for i in range(len(fr_references_gold))]

In [210]:
# print(np.max([getNamedEntityOverlap(fetchNamedEntities(fr_sents_gold[i], fr=True), fetchNamedEntities(transl_en_sents_gold[i], fr=True)) for i in range(len(gold_tuples[:100]))]))

In [26]:
# start = time.time()
# bad_indices = [all_sent_pairs.index(pair) for pair in all_sent_pairs if pair not in gold_tuples]
# fr_references_similar = [[fr_sents_only[all_sent_pairs[ind][1]-1].split()] for ind in bad_indices]
# fr_hypotheses_similar = [transl_en_sents[ind].split() for ind in bad_indices]
# end = time.time()
# print(end-start)

KeyboardInterrupt: 

In [211]:
corp_bleu_similar = nltk.translate.bleu_score.corpus_bleu(fr_references_similar, fr_hypotheses_similar)

In [212]:
sent_bleu_similar = [nltk.translate.bleu_score.sentence_bleu(fr_references_similar[i], fr_hypotheses_similar[i]) for i in range(len(fr_references_similar))]

In [275]:
k = 0
for score in sent_bleu_gold:
    if score > 0.1:
        k += 1
print(k)

print(len(sent_bleu_gold))

t = 0
for score in sent_bleu_similar:
    if score > 0.1:
        t += 1
print(t)

print(len(sent_bleu_similar))

5943
9029
7863
436021


In [272]:
print(5389/(5389+2460))
print(5389/(len(gold_tuples)))

0.6865842782520066
0.5931102795509575


0.6318110236220472

In [331]:
gold_cos_sims = [normalize(embed(fr_sents_gold[i])).dot(normalize(embed(transl_en_sents_gold[i]))) for i in range(1000)]

In [333]:
print(np.average(gold_cos_sims))

0.92529225


In [102]:
fr_sents_similar = []
for list_ in fr_references_similar:
    for sublist in list_:
        fr_sents_similar.append(" ".join(sublist))

NameError: name 'fr_references_similar' is not defined

In [214]:
transl_en_sents_similar = []
for list_ in fr_hypotheses_similar:
    transl_en_sents_similar.append(" ".join(list_))

In [215]:
print(len(fr_sents_similar))
print(len(transl_en_sents_similar))

436021
436021


In [347]:
similar_cos_sims = [normalize(embed(fr_sents_similar[i])).dot(normalize(embed(transl_en_sents_similar[i]))) for i in range(1000)]

In [349]:
print(np.average(similar_cos_sims))

0.23983933


In [ ]:
### Using LaBSE/SBERT cosine similarity as a filtering mechanism ###

In [ ]:
# start = time.time()
# cosine_similarities = [normalize(embed(fr_sents_only[all_sent_pairs[i][1]-1])).dot(normalize(embed(transl_en_sents[i]))) for i in range(len(transl_en_sents))]
# pd.DataFrame({'cosine_similarities': cosine_similarities}).to_csv('Data/cosine_similarities_LaBSE.csv', index=False)
# end = time.time()
# print(end-start)

In [130]:
### Using post-translation margin score as a filtering mechanism ###

In [79]:
all_cosine_similarities = pd.read_csv('Data/cosine_similarities_LaBSE.csv')
all_cosine_similarities = [float(sim) for sim in all_cosine_similarities['cosine_similarities']]

In [ ]:
### Using LASER cosine similarity as a filtering mechanism ###

In [28]:
# Importing dependencies

from laserembeddings import Laser
laser = Laser()

In [39]:
start = time.time()
laser_cos_sims = [normalize(laser.embed_sentences(fr_sents_only[all_sent_pairs[i][1]-1], lang='fr')).T.dot(normalize(laser.embed_sentences(transl_en_sents[i], lang='fr'))) for i in range(len(transl_en_sents[:100]))]
#pd.DataFrame({'cosine_similarities': laser_cos_sims}).to_csv('Data/cosine_similarities_LASER.csv', index=False)
end = time.time()
print(end-start)

1.5538535118103027


In [43]:
start = time.time()
fr_laser_emb = laser.embed_sentences([fr_sents_only[all_sent_pairs[i][1]-1] for i in range(len(all_sent_pairs))], lang='fr')
fr_laser_emb = [normalize(emb) for emb in fr_laser_emb]                
end = time.time()
print(end-start)

In [ ]:
start = time.time()
transl_en_laser_emb = laser.embed_sentences([transl_en_sents[i] for i in range(len(transl_en_sents))], lang='fr')
transl_en_laser_emb = [normalize(emb) for emb in transl_en_laser_emb]
end = time.time()
print(end-start)

In [49]:
start = time.time()
laser_cos_sims = [fr_laser_emb[i].dot(transl_en_laser_emb[i]) for i in range(len(transl_en_laser_emb))]
end = time.time()
print(end-start)

1.5644221305847168


In [50]:
pd.DataFrame({'cosine_similarities': laser_cos_sims}).to_csv('Data/cosine_similarities_LASER.csv', index=False)

In [51]:
laser_cosine_similarities = pd.read_csv('Data/cosine_similarities_LASER.csv')
laser_cosine_similarities = [float(sim) for sim in laser_cosine_similarities['cosine_similarities']]

In [92]:
# Average absolute difference between LASER and LaBSE cosine similarities
print(np.average([abs(laser_cosine_similarities[i]-all_cosine_similarities[i]) for i in range(len(laser_cosine_similarities))]))

0.07740991114879084


In [93]:
averaged_cosine_similarities = [((laser_cosine_similarities[i]+all_cosine_similarities[i])/2) for i in range(len(laser_cosine_similarities))]

In [110]:
### Combining margin/LaBSE/LASER similarity thresholds ###

THRESHOLD = 0.0
CS_THRESHOLD_LABSE = 0.84
CS_THRESHOLD_LASER = 0.85
CS_THRESHOLD_AVG = 0.84
updated_sent_pairs = []

for i in range(len(all_sent_pairs)):
    if all_margin_scores[i] > THRESHOLD and averaged_cosine_similarities[i] > CS_THRESHOLD_AVG: 
        updated_sent_pairs.append(all_sent_pairs[i])

print(computeF1(updated_sent_pairs, gold_tuples))

(0.8749247308910628, 0.8703038222793528, 0.8795949812898965)


In [ ]:
lexical_overlap_gold = [len(list((Counter(fr_sents_gold[i].split()) & Counter(transl_en_sents_gold[i].split())).elements())) for i in range(len(fr_sents_gold))]

In [123]:
print(sorted(lexical_overlap_gold))

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 

In [ ]:
lexical_overlap_similar = [len(list((Counter(fr_sents_similar[i].split()) & Counter(transl_en_sents_similar[i].split())).elements())) for i in range(len(fr_sents_similar))]

In [262]:
print(list((Counter(fr_sents_similar[200].split()) & Counter(transl_en_sents_similar[200].split())).elements()))

['«', 'comme', '».']


In [219]:
print(np.average(lexical_overlap_similar))
print(np.average(lexical_overlap_gold))

2.7943929306157274
436021
8.948720788570164


In [ ]:
print(len([l for l in lexical_overlap_similar if l >= 6]))
print(len(lexical_overlap_similar))
print(len([l for l in lexical_overlap_gold if l >= 6]))
print(len(lexical_overlap_gold))
print(len(lexical_overlap_gold)+len(lexical_overlap_similar))
print(len(all_sent_pairs))

In [ ]:
gold_inds = [all_sent_pairs.index(pair) for pair in all_sent_pairs if pair in gold_tuples]

In [ ]:
copy_lex_overlap_similar = lexical_overlap_similar
lexical_overlap_all = []
for i in range(len(lexical_overlap_gold)):
    copy_lex_overlap_similar.insert(gold_inds[i], lexical_overlap_gold[i])
lexical_overlap_all = copy_lex_overlap_similar
#lexical_overlap_all = [copy_lex_overlap_similar.insert(ind, lexical_overlap_gold[ind]) for ind in gold_inds]

In [165]:
print(len(lexical_overlap_all)-436021)
print(len(gold_tuples))
print(9218-len(gold_inds))
print(9218-len(lexical_overlap_gold))
print(len(gold_inds))

9218
9086
189
189
9029


In [ ]:
a = [lexical_overlap_all[ind] for ind in gold_inds]
b = [lexical_overlap_all[i] for i in range(len(all_sent_pairs)) if all_sent_pairs[i] not in gold_tuples]

In [187]:
print(len(a))
print(len(b))

9029
436021


In [ ]:
lexical_overlap_similar = b
los_copy = b

In [189]:
print(len(lexical_overlap_similar))

436021


In [27]:
lexical_overlap_all = [len(list((Counter(fr_sents_only[all_sent_pairs[i][1]-1].split()) & Counter(transl_en_sents[i].split())).elements())) for i in range(len(transl_en_sents))]

In [44]:
# Removing stopwords from lexical overlap list to create modified LO score #

fr_stopwords = open('Data/stopwords-fr.txt', 'r')

In [47]:
fr_stopwords = [word.strip('\n') for word in fr_stopwords]

In [50]:
# Obtaining lexical overlap between French and translated English sentences
# with stopwords removed

lexical_overlap_all_mod = []
for i in range(len(transl_en_sents)):
    c1 = Counter(fr_sents_only[all_sent_pairs[i][1]-1].split())
    c2 = Counter(transl_en_sents[i].split())
    overlap = list((c1 & c2).elements())
    mod_overlap = [e for e in overlap if e not in fr_stopwords]
    lexical_overlap_all_mod.append(len(mod_overlap))

In [92]:
### Using lexical overlap as a filtering mechanism ###

LO_THRESHOLD = 9
LO_THRESHOLD_MOD = 3
CS_THRESHOLD = 0.0
BLEU_THRESHOLD = 0.0
THRESHOLD = 0.85

updated_sent_pairs = []

for i in range(len(all_sent_pairs)):
    if all_margin_scores[i] > THRESHOLD and lexical_overlap_all_mod[i] >= LO_THRESHOLD_MOD and all_cosine_similarities[i] > CS_THRESHOLD:
        updated_sent_pairs.append(all_sent_pairs[i])

print(computeF1(updated_sent_pairs, gold_tuples))

(0.4457324986560443, 0.3025296038395285, 0.8463570327977108)


In [96]:
### Giving METEOR score a try instead of BLEU ###

meteor_scores_gold = [nltk.translate.meteor_score.single_meteor_score(fr_sents_gold[i], transl_en_sents_gold[i]) for i in range(len(fr_sents_gold))]

In [100]:
print(np.max(meteor_scores_gold))
print(np.average(meteor_scores_gold))
print(np.min(meteor_scores_gold))
print(len([score for score in meteor_scores_gold if score < 0.2]))

1.0375
0.5784231985366046
0.0
404


In [103]:
all_meteor_scores = [nltk.translate.meteor_score.single_meteor_score(fr_sents_only[all_sent_pairs[i][1]-1], transl_en_sents[i]) for i in range(len(all_sent_pairs))]

In [105]:
print(np.average(all_meteor_scores))
print(len([score for score in all_meteor_scores if score < 0.2]))

0.13697937502251198
361739


In [25]:
METEOR_THRESHOLD = 10
BLEU_THRESHOLD = 1000
CS_THRESHOLD = 0.75
LO_THRESHOD_MOD = 20
LO_THRESHOLD = 20

updated_sent_pairs = []

for i in range(len(all_sent_pairs)):
    if all_margin_scores[i] > THRESHOLD and \
    \
    (bleu_scores[i] > BLEU_THRESHOLD or \
     all_cosine_similarities[i] > CS_THRESHOLD or \
     lexical_overlap_all[i] > LO_THRESHOLD or \
     lexical_overlap_all_mod[i] > LO_THRESHOLD_MOD or \
     all_meteor_scores[i] > METEOR_THRESHOLD):
        
        updated_sent_pairs.append(all_sent_pairs[i])

print(computeF1(updated_sent_pairs, gold_tuples))

NameError: name 'bleu_scores' is not defined

In [247]:
print(len(updated_sent_pairs))
print(0.866*len(updated_sent_pairs))
print(len(fr_sents_gold)-7958)

9189
7957.674
1071


In [ ]:
### Using named entities as a filtering mechanism ###

In [35]:
#import spacy
#!python -m spacy download en_core_web_sm
#!python -m spacy download fr_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 14.7 MB 2.6 MB/s eta 0:00:01
  Created wheel for fr-core-news-sm: filename=fr_core_news_sm-2.3.0-py3-none-any.whl size=14718366 sha256=db8c7b45c2724cf4324ec2bfec25510aa7c47df06f1b90bee6326da0c3b8ef95
  Stored in directory: /scratch/1757124.1.csgpu/pip-ephem-wheel-cache-cqh4o4ox/wheels/cf/1e/f7/79485fa3afb8e3cb7490dedc3eb88b9fea2b4231a8209b77d1
Successfully built fr-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_sm')


In [57]:
import en_core_web_sm
import fr_core_news_sm

en_ne_model = en_core_web_sm.load(disable=['parser', 'tagger', 'textcat'])
fr_ne_model = fr_core_news_sm.load(disable=['parser', 'tagger', 'textcat'])

ImportError: cannot import name 'WhitespaceTokenizer' from 'spacy.tokenizer' (/usr2/collab/agjones/.local/lib/python3.7/site-packages/spacy/tokenizer.cpython-37m-x86_64-linux-gnu.so)

In [60]:
from spacy.tokens import Doc

# Copy-paste from SpaCy docs

class WhitespaceTokenizer(object):
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(' ')
        # All tokens 'own' a subsequent space character in this tokenizer
        spaces = [True] * len(words)
        return Doc(self.vocab, words=words, spaces=spaces)

In [62]:
en_ne_model.tokenizer = WhitespaceTokenizer(en_ne_model.vocab)
fr_ne_model.tokenizer = WhitespaceTokenizer(fr_ne_model.vocab)

In [26]:
# ne_model = 'usr2/collab/agjones/stanford-ner/classifiers/english.all.3class.distsim.crf.ser.gz'
# ne_jar = 'usr2/collab/agjones/stanford-ner/stanford-ner.jar'

In [83]:
def fetchNamedEntities(sent, en=False, fr=False):
    if en:
        doc = en_ne_model(sent)
    else:
        doc = fr_ne_model(sent)
    ne = [ent.text for ent in doc.ents]
    return ne

In [87]:
def getNamedEntityOverlap(ne_list_1, ne_list_2):
    return len(list((Counter(ne_list_1) & Counter(ne_list_2)).elements()))

In [79]:
print(getNamedEntityOverlap(fetchNamedEntities(fr_sents_only[all_sent_pairs[220362][1]-1]), fetchNamedEntities(transl_en_sents[220362])))

1


In [204]:
for sent in fr_sents_gold[:100]:
    print(fetchNamedEntities(sent, fr=True))

['Merkel']
['n’a']
['qu’aggraver']
[]
[]
[]
[]
[]
[]
[]
['d’une']
[]
[]
[]
['Etats-Unis']
['Les Etats-Unis']
['Ergogan.']
['district de Warangal', 'Pradesh.']
["l'Organisation mondiale du commerce"]
['L’Organisation mondiale du commerce aurait-elle']
[]
[]
[]
[]
[]
['États']
[]
['La Chine']
[]
[]
[]
[]
['Chine.']
['«\xa0promotion', 'sain\xa0».']
[]
[]
['Etats-Unis.']
['Nigeria.']
['l’échelle']
[]
[]
['Songez', 'British Petroleum', 'golfe du Mexique']
['États-nations']
['Fonds monétaire international']
[]
['John Snow', 'Londres.']
[]
[]
[]
["l'exception."]
[]
[]
[]
[]
[]
['Montesquieu', 'France.']
[]
[]
["l'épargne."]
[]
['Cependant,']
[]
[]
['n’a']
[]
[]
[]
[]
[]
[]
[]
[]
['Buffet']
[]
['2004,', 'Premier Ministre', 'Ahmed Nazif.']
[]
[]
['s’est', 'd’une']
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Malheureusement,']
['Roms', 'Europe']
[]
[]
[]
[]
["D'importants"]
['L’Europe', 'le Japon']
[]
[]


In [89]:
print(fetchNamedEntities(fr_sents_only[all_sent_pairs[220362][1]-1], fr=True))
print(fetchNamedEntities(en_sents_only[all_sent_pairs[220362][0]-1], en=True))

['Merkel']
['Merkel']


In [91]:
start = time.time()
NE_THRESHOLD = 0.4
updated_sent_pairs = []
for i in range(len(all_sent_pairs)):
    ne_en_sent = fetchNamedEntities(en_sents_only[all_sent_pairs[i][0]-1], en=True)
    ne_fr_sent = fetchNamedEntities(fr_sents_only[all_sent_pairs[i][1]-1], fr=True)
    if all_margin_scores[i] > THRESHOLD and (getNamedEntityOverlap(ne_en_sent, ne_fr_sent) / (len(ne_en_sent)+1e-8) > NE_THRESHOLD):
        updated_sent_pairs.append(all_sent_pairs[i])
end = time.time()
print(end-start)

print(computeF1(updated_sent_pairs, gold_tuples))

1783.4920241832733
(0.053263879441574825, 0.11253106141245109, 0.03488883997358574)


# Now, trying mining on the Tatoeba test sets:

# (1) en-kk, kk-en

In [14]:
# Loading in the test files
# For every pair of lines i,j in zip(file_a_lines, file_b_lines),
# a[i], b[j] represent a translation pair

with open('Data/tatoeba.kaz-eng.eng', 'r') as f:
    tatoeba_en_sents = f.read().splitlines()
with open ('Data/tatoeba.kaz-eng.kaz', 'r') as f:
    tatoeba_kk_sents = f.read().splitlines()

In [15]:
tatoeba_en_embeddings = embed(tatoeba_en_sents)
tatoeba_kk_embeddings = embed(tatoeba_kk_sents)

In [157]:
# Performing margin-based mining and obtaining accuracy scores

fwd_means, fwd_inds = directedMeansAndInds(tatoeba_en_embeddings, tatoeba_kk_embeddings, forward=True, k=4, batch_size=100)
bwd_means, bwd_inds = directedMeansAndInds(tatoeba_en_embeddings, tatoeba_kk_embeddings, backward=True, k=4, batch_size=100)

fwd_margin_scores = np.zeros(fwd_inds.shape)
for i in range(fwd_inds.shape[0]):
    for j in range(fwd_inds.shape[1]):
        tgt_ind = fwd_inds[i,j]
        margin_score = (tatoeba_en_embeddings[i].dot(tatoeba_kk_embeddings[tgt_ind]))# / np.average((fwd_means[i], bwd_means[tgt_ind]))
        fwd_margin_scores[i,j] = margin_score
best = np.zeros((fwd_inds.shape[0], 3))
best_inds = fwd_inds[np.arange(tatoeba_en_embeddings.shape[0]), fwd_margin_scores.argmax(axis=1)]
for i in range(fwd_inds.shape[0]):
    best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
    best[i] = ((i+1, best_inds[i]+1, best_score))
    
bwd_margin_scores = np.zeros(bwd_inds.shape)
for i in range(bwd_inds.shape[0]):
    for j in range(bwd_inds.shape[1]):
        tgt_ind = bwd_inds[i,j]
        margin_score = (tatoeba_kk_embeddings[i].dot(tatoeba_en_embeddings[tgt_ind]))# / np.average((bwd_means[i], fwd_means[tgt_ind]))
        bwd_margin_scores[i,j] = margin_score
bwd_best = np.zeros((bwd_inds.shape[0], 3))
best_inds = bwd_inds[np.arange(tatoeba_kk_embeddings.shape[0]), bwd_margin_scores.argmax(axis=1)]
for i in range(bwd_inds.shape[0]):
    best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
    #best_ind = bwd_inds[i][ind]
    bwd_best[i] = ((best_inds[i]+1, i+1, best_score))
    
fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

pairs_and_scores = []
pairs_and_scores = list(set(fwd_best) & set(bwd_best))
# for fwd_triplet, bwd_triplet in zip(fwd_best, bwd_best):
#     pairs_and_scores.append(fwd_triplet)
#     pairs_and_scores.append(bwd_triplet)

pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
# #pairs_and_scores = list(set(pairs_and_scores))
concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
#concat_pairs = list(dict.fromkeys(concat_pairs))
#concat_pairs = list(set(concat_pairs))
concat_pairs_int = []
for tup in concat_pairs:
    concat_pairs_int.append((int(tup[0]), int(tup[1])))
concat_pairs = concat_pairs_int

margin_scores = [triplet[2] for triplet in pairs_and_scores]

pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv('Data/sent_pairs_fwd_tat_en_kk_raw_cos.csv', index=False)
pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv('Data/sent_pairs_bwd_tat_en_kk_raw_cos.csv', index=False)
pd.DataFrame({'concat_pairs': concat_pairs}).to_csv('Data/concat_pairs_tat_en_kk_raw_cos.csv', index=False)
pd.DataFrame({'margin_scores': margin_scores}).to_csv('Data/margin_scores_tat_en_kk_raw_cos.csv', index=False)

In [158]:
all_margin_scores_tat_en_kk = pd.read_csv('Data/margin_scores_tat_en_kk_raw_cos.csv')
all_margin_scores_tat_en_kk = [float(score) for score in all_margin_scores_tat_en_kk['margin_scores']]
all_sent_pairs_tat_en_kk = pd.read_csv('Data/concat_pairs_tat_en_kk_raw_cos.csv')
all_sent_pairs_tat_en_kk = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs_tat_en_kk['concat_pairs']]
all_sent_pairs_tat_en_kk = [pair[0] for pair in all_sent_pairs_tat_en_kk]

In [28]:
def getAccuracyTatoeba(sent_pairs):
    return len([pair for pair in sent_pairs if pair[0]==pair[1]]) / len(sent_pairs)

In [161]:
print(computeF1(all_sent_pairs_tat_en_kk, ground_truth))

(0.9117647008902093, 0.9668615984216986, 0.8626086956521739)


In [64]:
# Translates using Helsinki-NLP open-source NMT system
def translateToKazakh(sents):
    for i in range(len(sents)):
        sents[i] = ">>kaz_Cyrl<< " + sents[i]
    tokenized = marian_tokenizer.prepare_seq2seq_batch(sents, return_tensors='pt')
    tokenized.to(device)
    translated = translation_model.generate(**tokenized)
    decoded = [marian_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    tokenized.to('cpu')
    return decoded

In [65]:
print(translateToKazakh(["This is a sentence. This is also a sentence. And this!"]))

['<pad> Мынау - сөйлем.']


In [20]:
NUM_EN_SENTS = len(tatoeba_en_sents)
BATCH_SIZE = 100
NUM_BATCHES = (NUM_EN_SENTS // BATCH_SIZE) + 1
i = 0
transl_tat_en_sents = []
start = time.time()
for i in range(NUM_BATCHES):
    transl_tat_en_sents.append(translateToKazakh(tatoeba_en_sents[i*BATCH_SIZE : (i+1)*BATCH_SIZE]))
    print("Completed batch {:} of {:}".format(i+1, NUM_BATCHES))
end = time.time()
print("Time taken: {:.3f}".format(end-start))

with open('Data/translated_english_sentences_tat.txt', 'w') as f:
    for batch in transl_tat_en_sents:
        for sent in batch:
            f.write("%s\n" % sent)

NameError: name 'marian_tokenizer' is not defined

In [56]:
with open('Data/translated_english_sentences_tat.txt', 'r') as f:
    transl_tat_en_sents = f.read().splitlines()

In [46]:
def mineSentencePairs(src_embs, tgt_embs, file_suffix, batch_size=100, num_neighbors=4):
    
    interval = 1e4

    print("Retrieving means in forward direction", flush=True)
    start = time.time()
    fwd_means, fwd_inds = directedMeansAndInds(src_embs, tgt_embs, forward=True, k=num_neighbors, batch_size=batch_size)
    end = time.time()
    print("Time taken to retrieve forward means: {:.2f} seconds".format(end-start), flush=True)
    
    print("Retrieving means in backward direction", flush=True)
    start = time.time()
    bwd_means, bwd_inds = directedMeansAndInds(src_embs, tgt_embs, backward=True, k=num_neighbors, batch_size=batch_size)
    end = time.time()
    print("Time taken to retrieve backward means: {:.2f} seconds".format(end-start), flush=True)
    
    print("Beginning mining in the forward direction", flush=True)
    fwd_margin_scores = np.zeros(fwd_inds.shape)
    for i in range(fwd_inds.shape[0]):
        if i % interval == 0 and i > 0:
            print("Completed pair {} of {} in the forward direction".format(i, fwd_inds.shape[0]), flush=True)
        for j in range(fwd_inds.shape[1]):
            tgt_ind = fwd_inds[i,j]
            margin_score = (src_embs[i].dot(tgt_embs[tgt_ind])) / np.average((fwd_means[i], bwd_means[tgt_ind]))
            fwd_margin_scores[i,j] = margin_score
    best = np.zeros((fwd_inds.shape[0], 3))
    best_inds = fwd_inds[np.arange(src_embs.shape[0]), fwd_margin_scores.argmax(axis=1)]
    for i in range(fwd_inds.shape[0]):
        best_score, ind = (np.max(fwd_margin_scores[i]), np.argmax(fwd_margin_scores[i]))
        best[i] = ((i+1, best_inds[i]+1, best_score))

    print("Mining in the forward direction complete", flush=True)
    
    print("Beginning mining in the backward direction", flush=True)
    bwd_margin_scores = np.zeros(bwd_inds.shape)
    for i in range(bwd_inds.shape[0]):
        if i % interval == 0 and i > 0:
            print("Completed pair {} of {} in the backward direction".format(i, bwd_inds.shape[0]), flush=True)
        for j in range(bwd_inds.shape[1]):
            tgt_ind = bwd_inds[i,j]
            margin_score = (tgt_embs[i].dot(src_embs[tgt_ind])) / np.average((bwd_means[i], fwd_means[tgt_ind]))
            bwd_margin_scores[i,j] = margin_score
    bwd_best = np.zeros((bwd_inds.shape[0], 3))
    best_inds = bwd_inds[np.arange(tgt_embs.shape[0]), bwd_margin_scores.argmax(axis=1)]
    for i in range(bwd_inds.shape[0]):
        best_score, ind = (np.max(bwd_margin_scores[i]), np.argmax(bwd_margin_scores[i]))
        #best_ind = bwd_inds[i][ind]
        bwd_best[i] = ((best_inds[i]+1, i+1, best_score))

    print("Mining in the backward direction complete", flush=True)
    
    fwd_best = [tuple(best[i]) for i in range(best.shape[0])]
    bwd_best = [tuple(bwd_best[i]) for i in range(bwd_best.shape[0])]

    pairs_and_scores = []
    pairs_and_scores = list(set(fwd_best) & set(bwd_best))
#     for fwd_triplet, bwd_triplet in zip(fwd_best, bwd_best):
#         pairs_and_scores.append(fwd_triplet)
#         pairs_and_scores.append(bwd_triplet)

    pairs_and_scores = list(dict.fromkeys(pairs_and_scores))
    # #pairs_and_scores = list(set(pairs_and_scores))
    concat_pairs = [(triplet[0], triplet[1]) for triplet in pairs_and_scores]
    #concat_pairs = list(dict.fromkeys(concat_pairs))
    #concat_pairs = list(set(concat_pairs))
    concat_pairs_int = []
    for tup in concat_pairs:
        concat_pairs_int.append((int(tup[0]), int(tup[1])))
    concat_pairs = concat_pairs_int

    margin_scores = [triplet[2] for triplet in pairs_and_scores]
    
    spf_path = 'Data/sent_pairs_fwd_{:}.csv'.format(file_suffix)
    spb_path = 'Data/sent_pairs_bwd_{:}.csv'.format(file_suffix)
    cct_path = 'Data/concat_pairs_{:}.csv'.format(file_suffix)
    mgn_path = 'Data/margin_scores_{:}.csv'.format(file_suffix)

    pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv(spf_path, index=False)
    pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv(spb_path, index=False)
    pd.DataFrame({'concat_pairs': concat_pairs}).to_csv(cct_path, index=False)
    pd.DataFrame({'margin_scores': margin_scores}).to_csv(mgn_path, index=False)

    all_margin_scores = pd.read_csv(mgn_path)
    all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
    all_sent_pairs = pd.read_csv(cct_path)
    all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
    all_sent_pairs = [pair[0] for pair in all_sent_pairs]
                                    
    return concat_pairs, margin_scores

In [57]:
all_sent_pairs_en_kk_T, all_margin_scores_en_kk_T = mineSentencePairs(embed(transl_tat_en_sents), 
                                                                      tatoeba_kk_embeddings,
                                                                      'tat_en_kk_T')

In [58]:
print(computeF1(all_sent_pairs_en_kk_T, ground_truth))

(0.7910592759166485, 0.8964757709053639, 0.7078260869565217)


In [67]:
print(transl_tat_en_sents[:10])
print(tatoeba_kk_sents[:10])

['Өткен жылы ең жоғары мектепті біттім.', 'Жаңа жылың қуаныш!', 'Дегенім жоқ.', 'Жалғыз қарым-қатынас.', 'Сені сындырып кетмесе деймін.', 'Оны бір күні оқып алды.', 'Маған өлтір!', 'Занар Амантавенс балалары үшін мектеп саласындағы әдеби идеяларды қоғамдатады.', 'Бүгінгі Дүйсенбі.', 'Негізінде ол нағыз адам.']
['Мен былтыр орта мектепті бітірдім.', 'Жаңа жылыңыз құтты болсын.', 'Бiлгiм келмейдi.', 'Қарыз қатынас бұзады.', 'Сізді бөлмедім деп үміттенемін.', 'Ол кітапты бір күнде оқып бітірді.', 'Өлтірдің ғой мені.', 'Жанар Амантаевна біздің балалардың жазғы демалысын ұйымдастыруда жақсы ой тастайды.', 'Бүгін Дүйсенбі.', 'Менің пікірімше, ол адал адам.']


In [68]:
# Trying translating in the other direction . . .

translation_model_name = 'Helsinki-NLP/opus-mt-trk-en'
marian_tokenizer = MarianTokenizer.from_pretrained(translation_model_name)
translation_model = MarianMTModel.from_pretrained(translation_model_name)
translation_model.cuda().half() # Puts translation model on GPU

Downloading:   0%|          | 0.00/305M [00:00<?, ?B/s]

MarianMTModel(
  (model): BartModel(
    (shared): Embedding(61709, 512, padding_idx=61708)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(61709, 512, padding_idx=61708)
      (embed_positions): SinusoidalPositionalEmbedding(512, 512)
      (layers): ModuleList(
        (0): EncoderLayer(
          (self_attn): Attention(
            (k_proj): Linear(in_features=512, out_features=512, bias=True)
            (v_proj): Linear(in_features=512, out_features=512, bias=True)
            (q_proj): Linear(in_features=512, out_features=512, bias=True)
            (out_proj): Linear(in_features=512, out_features=512, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
          (fc1): Linear(in_features=512, out_features=2048, bias=True)
          (fc2): Linear(in_features=2048, out_features=512, bias=True)
          (final_layer_norm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        )
        (1): EncoderLayer

In [69]:
def translateFromKazakh(sents):
    for i in range(len(sents)):
        sents[i] = ">>kaz_Cyrl<< " + sents[i]
    tokenized = marian_tokenizer.prepare_seq2seq_batch(sents, return_tensors='pt')
    tokenized.to(device)
    translated = translation_model.generate(**tokenized)
    decoded = [marian_tokenizer.decode(t, skip_special_tokens=True) for t in translated]
    tokenized.to('cpu')
    return decoded

In [71]:
NUM_EN_SENTS = len(tatoeba_kk_sents)
BATCH_SIZE = 100
NUM_BATCHES = (NUM_EN_SENTS // BATCH_SIZE) + 1
i = 0
transl_tat_kk_sents = []
start = time.time()
for i in range(NUM_BATCHES):
    transl_tat_kk_sents.append(translateFromKazakh(tatoeba_kk_sents[i*BATCH_SIZE : (i+1)*BATCH_SIZE]))
    print("Completed batch {:} of {:}".format(i+1, NUM_BATCHES))
end = time.time()
print("Time taken: {:.3f}".format(end-start))

with open('Data/translated_kazakh_sentences_tat.txt', 'w') as f:
    for batch in transl_tat_kk_sents:
        for sent in batch:
            f.write("%s\n" % sent)

with open('Data/translated_kazakh_sentences_tat.txt', 'r') as f:
    transl_tat_kk_sents = f.read().splitlines()

In [72]:
all_sent_pairs_en_kk_T2, all_margin_scores_en_kk_T2 = mineSentencePairs(embed(transl_tat_kk_sents), 
                                                                      tatoeba_kk_embeddings,
                                                                      'tat_en_kk_T2')
print(computeF1(all_sent_pairs_en_kk_T2, ground_truth))

(0.5583982156282659, 0.7746913580007811, 0.4365217391304348)


In [141]:
usp = []
for i in range(len(all_sent_pairs_en_kk)):
    if all_margin_scores_en_kk[i] > 1.25:
        usp.append(all_sent_pairs_en_kk[i])
        
print(computeF1(usp, ground_truth))

(0.7869198264423436, 0.9999999999731903, 0.648695652173913)


In [31]:
from googleapiclient.discovery import build
import getpass

In [38]:
# Getting our Google Translate API key from Google Cloud (this is one-time and user-specific)
APIKEY = getpass.getpass()

········


In [39]:
service = build('translate', 'v2', developerKey=APIKEY)

Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/s

In [84]:
def translateGoogle(sentence):
    return (((service.translations().list(source='kk', target='en', q=[sentence]).execute())['translations'])[0])['translatedText']

In [85]:
transl_tat_kk_sents_goog = [translateGoogle(sent) for sent in tatoeba_kk_sents]

In [86]:
all_sent_pairs_en_kk_goog2, all_margin_scores_en_kk_goog2 = mineSentencePairs(embed(transl_tat_kk_sents_goog), 
                                                                      tatoeba_en_embeddings,
                                                                      'tat_en_kk_goog2')
print(getAccuracyTatoeba(all_sent_pairs_en_kk_goog2))

0.9664804469273743


In [69]:
r = len(all_sent_pairs_en_kk_goog)/575

0.9147826086956522


In [44]:
ground_truth = [(i+1,i+1) for i in range(len(tatoeba_en_sents))]

In [87]:
print(computeF1(all_sent_pairs_en_kk_goog2, ground_truth))

(0.9334532324075165, 0.9664804469093765, 0.9026086956521739)


In [45]:
print(len(transl_tat_en_sents_goog))

575


In [147]:
all_sent_pairs_en_kk_goog2, all_margin_scores_en_kk_goog2 = mineSentencePairs(embed(transl_tat_kk_to_en), 
                                                                      tatoeba_en_embeddings,
                                                                      'tat_en_kk_goog2')
print(getAccuracyTatoeba(all_sent_pairs_en_kk_goog2))

0.9664804469273743


In [148]:
print(computeF1(all_sent_pairs_en_kk_goog2, ground_truth))

(0.9334532324075165, 0.9664804469093765, 0.9026086956521739)


In [92]:
with open('Data/transl_tat_kk_to_en.txt', 'w') as f:
    for sent in transl_tat_en_sents_goog2:
        f.write("%s\n" % sent)

In [93]:
with open('Data/transl_tat_en_to_kk.txt', 'w') as f:
    for sent in transl_tat_en_sents_goog:
        f.write("%s\n" % sent)

In [95]:
with open('Data/transl_tat_en_to_kk.txt', 'r') as f:
    transl_tat_en_to_kk = f.read().splitlines()

In [97]:
with open('Data/transl_tat_kk_to_en.txt', 'r') as f:
    transl_tat_kk_to_en = f.read().splitlines()

In [144]:
print(computeF1(all_sent_pairs_en_kk_goog, ground_truth))

(0.9706144206398394, 0.9945255474271072, 0.9478260869565217)


In [174]:
# merged_pairs = []
# for pair1, pair2 in zip(all_sent_pairs_en_kk_goog2, all_sent_pairs_tat_en_kk):
#     merged_pairs.append(pair1)
#     merged_pairs.append(pair2)

In [177]:
# merged_set = set(merged_pairs)

In [193]:
# merged_pairs_new = []
# for i in range(len(all_sent_pairs_tat_en_kk)):
#     for j in range(len(all_sent_pairs_en_kk_goog2)):
#         if all_sent_pairs_tat_en_kk[i][0]==all_sent_pairs_en_kk_goog2[j][0]:
#             if all_margin_scores_tat_en_kk[i] >= all_margin_scores_en_kk_goog2[j]:
#                 merged_pairs_new.append(all_sent_pairs_tat_en_kk[i])
#             else:
#                 merged_pairs_new.append(all_sent_pairs_en_kk_goog2[j])

In [75]:
print(computeF1(list(set(all_sent_pairs_en_kk)&set(all_sent_pairs_en_kk_T)&set(all_sent_pairs_en_kk_T2)), ground_truth))

(0.5142118824782165, 0.9999999999497488, 0.34608695652173915)


In [76]:
pairs1 = set(all_sent_pairs_en_kk)
pairs2 = set(all_sent_pairs_en_kk_T)
pairs3 = set(all_sent_pairs_en_kk_T2)
int1,int2,int3 = pairs1&pairs2, pairs1&pairs3, pairs2&pairs3
majority_vote = list(set(int1|int2|int3))
print(computeF1(majority_vote, ground_truth))

(0.8622047194879371, 0.9931972788890431, 0.7617391304347826)


In [ ]:
print(len(majority_vote))

In [54]:
# Obtaining the minimum margin score for a recalled ground-truth pair
ground_truth_ms = []
for i in range(len(all_margin_scores_en_kk_goog2)):
    if all_sent_pairs_en_kk_goog2[i][0]==all_sent_pairs_en_kk_goog2[i][1]:
        ground_truth_ms.append(all_margin_scores_en_kk_goog2[i])
print(min(ground_truth_ms)) # We'll set this as our threshold when mining sentence pairs for NMT

1.0468919277191162


In [55]:
# TESTING THE ABOVE METHOD ON KHMER TATOEBA DATA
# Benchmark to beat: 83.2 (https://arxiv.org/pdf/2004.09813.pdf)

# Loading in the test files
# For every pair of lines i,j in zip(file_a_lines, file_b_lines),
# a[i], b[j] represent a translation pair
with open('Data/tatoeba.khm-eng.eng', 'r') as f:
    tatoeba_en_sents_2 = f.read().splitlines()
with open ('Data/tatoeba.khm-eng.khm', 'r') as f:
    tatoeba_khm_sents = f.read().splitlines()
    
tatoeba_en_embeddings_2 = embed(tatoeba_en_sents_2)
tatoeba_khm_embeddings = embed(tatoeba_khm_sents)

In [56]:
all_sent_pairs_en_khm, all_margin_scores_en_khm = mineSentencePairs(tatoeba_en_embeddings_2, 
                                                                      tatoeba_khm_embeddings,
                                                                      'tat_en_khm')
print(getAccuracyTatoeba(all_sent_pairs_en_khm))

0.93026941362916


In [68]:
print(getAccuracyTatoeba(list(set(all_sent_pairs_en_khm)&set(all_sent_pairs_en_km_goog2)&set(all_sent_pairs_en_km_goog))))

0.9764521193092621


In [72]:
# Obtaining the minimum margin score for a recalled ground-truth pair
ground_truth_ms_2 = []
for i in range(len(all_margin_scores_en_km_goog2)):
    if all_sent_pairs_en_km_goog2[i][0]==all_sent_pairs_en_km_goog2[i][1]:
        ground_truth_ms_2.append(all_margin_scores_en_km_goog2[i])
print(np.average(ground_truth_ms_2)) # We'll set this as our threshold when mining sentence pairs for NMT

1.3639547663963645


In [26]:
# Derry wants me to try using word-by-word translation again, this time on Tatoeba . . .

kk_en_dict = pd.read_csv("Data/new_merged_dict.kk", names=['entries'])

In [34]:
new_kk_en_dict = {str(entry).split("\t")[0]:str(entry).split("\t")[1] for entry in kk_en_dict['entries'] 
                  if len(str(entry).split("\t"))==2}

In [39]:
wbw_transl_tat_kk = []
for sent in tatoeba_kk_sents:
    transl = ""
    for word in sent.split():
        if word in new_kk_en_dict:
            transl += new_kk_en_dict[word] + " "
        else:
            transl += word + " "
    wbw_transl_tat_kk.append(transl)

In [46]:
all_sent_pairs_tat_en_kk_wbw1, all_margin_scores_tat_en_kk_wbw1 = mineSentencePairs(embed(wbw_transl_tat_kk), 
                                                                                       tatoeba_en_embeddings,
                                                                                       "tat_en_kk_wbw1")
print(computeF1(all_sent_pairs_tat_en_kk_wbw1, ground_truth))

(0.8494726700205084, 0.9465811965609705, 0.7704347826086957)


In [47]:
en_kk_dict = {str(entry).split("\t")[1]:str(entry).split("\t")[0] for entry in kk_en_dict['entries'] 
                  if len(str(entry).split("\t"))==2}

In [48]:
wbw_transl_tat_en = []
for sent in tatoeba_en_sents:
    transl = ""
    for word in sent.split():
        if word in en_kk_dict:
            transl += en_kk_dict[word] + " "
        else:
            transl += word + " "
    wbw_transl_tat_en.append(transl)

In [49]:
all_sent_pairs_tat_en_kk_wbw2, all_margin_scores_tat_en_kk_wbw2 = mineSentencePairs(embed(wbw_transl_tat_en), 
                                                                                       tatoeba_kk_embeddings,
                                                                                       "tat_en_kk_wbw2")
print(computeF1(all_sent_pairs_tat_en_kk_wbw2, ground_truth))

(0.876064328048394, 0.9605809128431415, 0.8052173913043478)


# AT LAST, mining pseudoparallel sentences from the en-kk data

In [ ]:
# Reading in and cleaning the data

# Importing the English-Kazakh comparable corpus (Wikipedia)
column_names = ['Kazakh', 'English']
en_kk_comparable_corpus = pd.read_csv('/project/statnlp/ajones/kazakh_full_paragraph.csv', names=column_names).drop_duplicates().dropna()

# Putting the sentences into lists
kazakh_sentences = en_kk_comparable_corpus.Kazakh.to_list()
english_sentences = en_kk_comparable_corpus.English.to_list()

kazakh_sentences = [str(sentence) for sentence in kazakh_sentences]
english_sentences = [str(sentence) for sentence in english_sentences]

# Preprocessing the data
split_english_sentences = [nltk.tokenize.sent_tokenize(sentence) for sentence in english_sentences]
split_kazakh_sentences = [nltk.tokenize.sent_tokenize(sentence) for sentence in kazakh_sentences]

def clean_english_sentence(data):
    data = re.sub(r"https?://[A-Za-z0-9./]+", " ", data) # gets rid of URLs
    data = re.sub(r"[^A-Za-z.!?',;:—0-9]", " ", data) # gets rid of non-standard characters
    data = re.sub(r" +", " ", data) # eliminates superfluous white space
    return data

def clean_english_data(data):
    new_data = []
    for paragraph in data:
        paragraph = [clean_english_sentence(sentence) for sentence in paragraph]
        paragraph = [sentence.replace('a href', '') for sentence in paragraph]
        paragraph = [sentence.replace('href', '') for sentence in paragraph]
        new_data.append(paragraph)
    return new_data

cleaned_english_sentences = clean_english_data(split_english_sentences)

def clean_kazakh_sentence(data):
    data = re.sub(r"https?://[A-Za-z0-9./]+", " ", data) # gets rid of URLs
    data = re.sub(r"[^.!?',;:—АаӘәБбВвГгҒғДдЕеЁёЖжЗзИиЙйКкҚқЛлМмНнҢңОоӨөПпРрСсТтУуҰұҮүФфХхҺһЦцЧчШшЩщЪъЫыІіЬьЭэЮюЯя]", " ", data) # gets rid of non-standard characters
    data = re.sub(r" +", " ", data) # eliminates superfluous white space
    return data

def clean_kazakh_data(data):
    new_data = []
    for paragraph in data:
        paragraph = [clean_kazakh_sentence(sentence) for sentence in paragraph]
        paragraph = [sentence.replace('a href', '') for sentence in paragraph]
        paragraph = [sentence.replace('href', '') for sentence in paragraph]
        new_data.append(paragraph)
    return new_data

cleaned_kazakh_sentences = clean_kazakh_data(split_kazakh_sentences)

for paragraph in cleaned_kazakh_sentences:
    for sentence in paragraph:
        if sentence.find('Тағы қара') != -1:
            paragraph.remove(sentence)

# There are some empty lists in the Kazakh sentences, so we'll just insert an empty
# string into them (which can be removed after mining—we want to preserve document-level
# alignment beforehand)
for arr in cleaned_kazakh_sentences:
    if len(arr)==0:
        arr.append('')
            
# new_cleaned_kazakh_sentences = []
# for i in range(len(cleaned_kazakh_sentences)):
#     if len(cleaned_kazakh_sentences[i]) > 1:
#         new_cleaned_kazakh_sentences.append(cleaned_kazakh_sentences[i])

# cleaned_kazakh_sentences = new_cleaned_kazakh_sentences

# cleaned_english_sentences_new = []
# for par in cleaned_english_sentences:
#     for sent in par:
#         cleaned_english_sentences_new.append(sent)
# cleaned_english_sentences = cleaned_english_sentences_new

# cleaned_kazakh_sentences_new = []
# for par in cleaned_kazakh_sentences:
#     for sent in par:
#         cleaned_kazakh_sentences_new.append(sent)
# cleaned_kazakh_sentences = cleaned_kazakh_sentences_new

In [ ]:
print(len(cleaned_english_sentences)==len(cleaned_kazakh_sentences))

In [ ]:
# # First, mining the "standard" way
# start = time.time()
# all_sent_pairs_en_kk_old_method, all_margin_scores_en_kk_old_method = mineSentencePairs(embed(cleaned_english_sentences),
#                                                                                        embed(cleaned_kazakh_sentences),
#                                                                                        'en_kk_old_method')
# end = time.time()
# print(end-start)

In [ ]:
start = time.time()
cleaned_english_embeddings = embed(cleaned_english_sentences)
end = time.time()
print("Time taken to embed {} sentences: {.2f} seconds".format(len(cleaned_english_embeddings), end-start))

In [ ]:
start = time.time()
cleaned_kazakh_embeddings = embed(cleaned_kazakh_sentences)
end = time.time()
print("Time taken to embed {} sentences: {:.2f} seconds".format(len(cleaned_kazakh_embeddings), end-start))

In [ ]:
# Whoops . . . converting our English embeddings back to a document-separated format
cleaned_en_embs_by_doc = []
j = 0
for doc in cleaned_english_sentences:
    i = 0
    doc_embs = np.zeros((len(doc), cleaned_english_embeddings[0].shape[0]), dtype=np.float32)
    while i < len(doc):
        doc_embs[i] = cleaned_english_embeddings[j]
        j += 1
        i += 1
    cleaned_en_embs_by_doc.append(doc_embs)

In [ ]:
print(len(cleaned_en_embs_by_doc)==len(cleaned_english_sentences))

In [ ]:
# First, mining the "standard" way
# Because the search space is so large, and our computational resources are limited, we'll restrict the 
# search space to the document level, as opposed to mining globally

start = time.time()

i = 0
MOD = 1e3
all_sent_pairs_en_kk_old_method, all_margin_scores_en_kk_old_method = [], []
for en_doc, kk_doc in zip(cleaned_en_embs_by_doc, cleaned_kazakh_sentences):
    i += 1
    if i % MOD == 0:
        print("Completed document {} of {}".format(i, len(cleaned_kazakh_sentences)), flush=True)
    doc_sent_pairs, doc_margin_scores = mineSentencePairs(en_doc, embed(kk_doc), '')
    all_sent_pairs_en_kk_old_method.append(doc_sent_pairs)
    all_margin_scores_en_kk_old_method.append(doc_margin_scores)
    
# file_suffix = 'en_kk_old_method'
    
# spf_path = 'Data/sent_pairs_fwd_{:}.csv'.format(file_suffix)
# spb_path = 'Data/sent_pairs_bwd_{:}.csv'.format(file_suffix)
# cct_path = 'Data/concat_pairs_{:}.csv'.format(file_suffix)
# mgn_path = 'Data/margin_scores_{:}.csv'.format(file_suffix)
    
# pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv(spf_path, index=False)
# pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv(spb_path, index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv(cct_path, index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv(mgn_path, index=False)

# all_margin_scores = pd.read_csv(mgn_path)
# all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
# all_sent_pairs = pd.read_csv(cct_path)
# all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
# all_sent_pairs = [pair[0] for pair in all_sent_pairs]

end = time.time()
print("Time taken to mine all sentence pairs: {:.2f} seconds".format(end-start))

In [ ]:
# Filtering using a set threshold

MARGIN_THRESHOLD = 1.35
updated_sent_pairs_en_kk_old_method = []
for i in range(len(all_sent_pairs_en_kk_old_method)):
    doc_pairs = []
    for j in range(len(all_sent_pairs_en_kk_old_method[i])):
        if all_margin_scores_en_kk_old_method[i][j] > MARGIN_THRESHOLD:
            doc_pairs.append(all_sent_pairs_en_kk_old_method[i][j])
    updated_sent_pairs_en_kk_old_method.append(doc_pairs)                   

In [ ]:
print(len(updated_sent_pairs_en_kk_old_method))

In [ ]:
print(sum([len(l) for l in updated_sent_pairs_en_kk_old_method]))

In [ ]:
mined_en_sents_old, mined_kk_sents_old = [], []
for i in range(len(updated_sent_pairs_en_kk_old_method)):
    for j in range(len(updated_sent_pairs_en_kk_old_method[i])):
        pair = updated_sent_pairs_en_kk_old_method[i][j]
        mined_en_sents_old.append(cleaned_english_sentences[i][pair[0]-1])
        mined_kk_sents_old.append(cleaned_kazakh_sentences[i][pair[1]-1])

In [ ]:
print(len(mined_en_sents_old)==len(mined_kk_sents_old))

In [123]:
print(mined_en_sents_old[:10])

["In August 634 the dying Caliph  Abu 20Bakr Abu Bakr a called in 'Abd al Rahman and  Uthman Uthman a to inform them that he had designated  Umar 20ibn 20al Khattab Umar ibn al Khattab a as successor.", 'He was buried on a hill to the north east of present day  Amman Amman a ,  Jordan Jordan a .', 'He is known for being one of the  Hadith 20of 20the 20ten 20promised 20paradise Ten Promised Paradise a .', 'At the time of the decision, therefore, the melody of O sole mio had not yet—as had been widely supposed—entered into the  public 20domain public domain a in any country that was a party to the  Berne 20Convention Berne Convention a during the relevant period.', "When performing it in concert in the mid 1970s, Elvis would explain the origin of It's Now Or Never and have singer Sherrill Nielsen perform a few lines of the original Neapolitan version before commencing with his version.", "is the Neapolitan equivalent of standard Italian ' and translates literally as my sun or my sunshine

In [114]:
print(mined_kk_sents_old[:10])

['халифа Әбу Бәкір Аллаһның оған игілігі мен сәлемі болсын жылы қайтыс болғанға дейін кейін Абдуррахманды шақырып Умар ибн әл Хаттаб ты өзінің орынбасары ретінде тағайындайтынын мәлімдейді.', 'Абдуррахман ибн Ауф Ислам дінін қабылдаған ең алғашқы сегіз адамдардың бірі болған.', 'Абдуррахман ибн Ауф басқа аты Абдел Рахман ибн Ғауф.', 'Абдуррахман ибн Ауф Абдуррахман ибн Ауф, көрнекті сахабалардың бірі болған.', 'Хиджраның жылы Осман ибн Аффан ның билігі кезінде Левант қайтыс болады, бүгінгі күнгі Амман , Иордания қаласының солтүстік беткейінде жерленеді.', 'Жаннатқа қабылданған он адамның бірі.', "Шешім кезінде, 'О соле мио мелодиясы емес — бұл кеңінен деп болжам бойынша, енді кез келген елде ол қоғамдық игілік болып табылады, Берн Конвенциясына қатысушы кезеңге тиісті.", " жылдардың ортасында орындау кезінде оның концертінде Элвис ' Қазір немесе ешқашан шығу тегін түсіндіріп еді мен әнші Шерил Нильсен өз нұсқасының алдында итальяндық нұсқасының түпнұсқаның бірнеше жолдарын орындады.", 

In [124]:
print(len(mined_en_sents_old))

217667


In [ ]:
# Writing the sentence pairs mined with the "old" method to a file
pd.DataFrame({'source': mined_en_sents_old, 'target': mined_kk_sents_old}).to_csv('Data/en_kk_mined_old_method-1.35.csv', index=False)

In [ ]:
# Now, mining using the new method (bidirectional translation + intersection)

In [27]:
# Translating English sentences to Kazakh

APIKEY = getpass.getpass()
service = build('translate', 'v2', developerKey=APIKEY)

def translateGoogle(sentence):
    return (((service.translations().list(source='en', target='kk', q=[sentence]).execute())['translations'])[0])['translatedText']

········


Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 33, in <module>
    from oauth2client.contrib.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.contrib.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/file_cache.py", line 37, in <module>
    from oauth2client.locked_file import LockedFile
ModuleNotFoundError: No module named 'oauth2client.locked_file'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/site-packages/googleapiclient/discovery_cache/__init__.py", line 44, in autodetect
    from . import file_cache
  File "/share/pkg.7/python3/3.7.9/install/lib/python3.7/s

In [17]:
start = time.time()
en_to_kk_sents = []
for doc in cleaned_english_sentences[3:5]:
    doc_sents = []
    for sent in doc:
        doc_sents.append(translateGoogle(sent))
    en_to_kk_sents.append(doc_sents)
end = time.time()
print(end-start)

1.6862187385559082


In [134]:
print(len(cleaned_english_sentences[3])+len(cleaned_english_sentences[4]))

19


In [135]:
print(sum([len(doc) for doc in cleaned_english_sentences]))

4488396


In [138]:
4488396*2.294/19/3600/24

6.272161564327486

In [139]:
x = []
for doc in cleaned_english_sentences:
    for sent in doc:
        x.append(sent)

In [140]:
start = time.time()
a= [translateGoogle(sent) for sent in x[:100]]
end = time.time()
print(end-start)

29.82867455482483


In [144]:
with open('/project/statnlp/ajones/english_sents_from_cc.txt', 'w') as f:
    for sent in x:
        f.write("%s\n" % sent)

In [145]:
with open('/project/statnlp/ajones/english_sents_from_cc.txt', 'r') as f:
    eng_sents_from_cc = f.read().splitlines()

In [146]:
print(len(eng_sents_from_cc))

4488396


In [147]:
y = []
for doc in cleaned_kazakh_sentences:
    for sent in doc:
        y.append(sent)

In [148]:
with open('/project/statnlp/ajones/kazakh_sents_from_cc.txt', 'w') as f:
    for sent in y:
        f.write("%s\n" % sent)

In [149]:
with open('/project/statnlp/ajones/kazakh_sents_from_cc.txt', 'r') as f:
    kaz_sents_from_cc = f.read().splitlines()

In [151]:
!pip install google.cloud

Defaulting to user installation because normal site-packages is not writeable


In [3]:
# Using batch translation for faster processing

# Code source: https://cloud.google.com/translate/docs/advanced/batch-translation#translate_v3_batch_translate_text-python

from google.cloud import translate
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS']="Data/mineral-weaver-301202-eeca7bc928a3.json"

def batch_translate_text(
    input_uri="gs://YOUR_BUCKET_ID/path/to/your/file.txt",
    output_uri="gs://YOUR_BUCKET_ID/path/to/save/results/",
    project_id="YOUR_PROJECT_ID",
    timeout=180
):
    """Translates a batch of texts on GCS and stores the result in a GCS location."""

    client = translate.TranslationServiceClient()

    location = "us-central1"
    # Supported file types: https://cloud.google.com/translate/docs/supported-formats
    gcs_source = {"input_uri": input_uri}

    input_configs_element = {
        "gcs_source": gcs_source,
        "mime_type": "text/plain",  # Can be "text/plain" or "text/html".
    }
    gcs_destination = {"output_uri_prefix": output_uri}
    output_config = {"gcs_destination": gcs_destination}
    parent = f"projects/{project_id}/locations/{location}"

    # Supported language codes: https://cloud.google.com/translate/docs/language
    operation = client.batch_translate_text(
        request={
            "parent": parent,
            "source_language_code": "kk",
            "target_language_codes": ["en"],  # Up to 10 language codes here.
            "input_configs": [input_configs_element],
            "output_config": output_config,
        }
    )

    print("Waiting for operation to complete...")
    response = operation.result(timeout)

    print("Total Characters: {}".format(response.total_characters))
    print("Translated Characters: {}".format(response.translated_characters))

In [ ]:
start = time.time()
batch_translate_text(input_uri="gs://translation-for-bitext-mining/kazakh_sents_from_cc.txt/",
                    output_uri="gs://translated-cc-out/",
                    project_id="mineral-weaver-301202",
                    timeout=100000)
end = time.time()
print(end-start)

Waiting for operation to complete...


In [47]:
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])

usr2/collab/agjones/Data/mineral-weaver-301202-eeca7bc928a3.json


In [43]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="usr2/collab/agjones/Data/mineral-weaver-301202-eeca7bc928a3.json"

In [48]:
!export GOOGLE_APPLICATION_CREDENTIALS="usr2/collab/agjones/Data/mineral-weaver-301202-eeca7bc928a3.json"

In [24]:
def explicit():
    from google.cloud import storage

    # Explicitly use service account credentials by specifying the private key
    # file.
    storage_client = storage.Client.from_service_account_json(
        'Data/mineral-weaver-301202-a2309a626c27.json')

    # Make an authenticated API request
    buckets = list(storage_client.list_buckets())
    print(buckets)

In [55]:
from google.cloud import storage
storage_client = storage.Client()
buckets = list(storage_client.list_buckets())

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b?project=mineral-weaver-301202&projection=noAcl&prettyPrint=false: alex-s-account@mineral-weaver-301202.iam.gserviceaccount.com does not have storage.buckets.list access to the Google Cloud project.

In [16]:
explicit()

Forbidden: 403 GET https://storage.googleapis.com/storage/v1/b?project=mineral-weaver-301202&projection=noAcl&prettyPrint=false: alex-s-account@mineral-weaver-301202.iam.gserviceaccount.com does not have storage.buckets.list access to the Google Cloud project.

In [57]:
bucket = storage_client.bucket("translation-for-bitext-mining")

In [14]:
pip install google-cloud-storage

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 96 kB 3.8 MB/s eta 0:00:011
     |████████████████████████████████| 75 kB 2.1 MB/s  eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [6]:
print(end-start)

1677.345908164978


In [195]:
# Mining from a subset of the en-kk data in order to obtain the "pseudo-gold-standard" threshold

# Shoot for a subset of documents containing ~10K sentences
print(sum([len(doc) for doc in cleaned_english_sentences[2000:7000]]))

182022


In [41]:
def translateGoogle(sentences: list, src_code: str, tgt_code: str):
    transl_doc = (((service.translations().list(source=src_code, target=tgt_code, q=sentences, format='text').execute())))#['translations'])[0])['translatedText']
    return [sent['translatedText'] for sent in transl_doc['translations']]

In [42]:
print(translateGoogle(["this is a sentence"], "en", "af"))

["dit is 'n sin"]


In [197]:
start = time.time()
transl_en_sents_goog = []
bad_indices = []
i = 2000
for doc in cleaned_english_sentences[2000:7000]:
    try:
        transl_en_sents_goog.append(translateGoogle(doc, "en", "kk"))
        i += 1
        if i % 200 == 0:
            print("Finished doc at index {}".format(i), flush=True)
    except:
        bad_indices.append(i)
        i += 1
        continue
end = time.time()
print(end-start)

Finished doc at index 2200
Finished doc at index 2400
Finished doc at index 2600
Finished doc at index 2800
Finished doc at index 3000
Finished doc at index 3200
Finished doc at index 3400
Finished doc at index 3800
Finished doc at index 4000
Finished doc at index 4400
Finished doc at index 4600
Finished doc at index 4800
Finished doc at index 5000
Finished doc at index 5200
Finished doc at index 5400
Finished doc at index 5600
Finished doc at index 5800
Finished doc at index 6000
Finished doc at index 6200
Finished doc at index 6400
Finished doc at index 6600
Finished doc at index 6800
Finished doc at index 7000
2403.6892714500427


In [198]:
# Number of translated sents we ended up with
print(sum([len(d) for d in transl_en_sents_goog]))

84759


In [199]:
start = time.time()
transl_kk_sents_goog = []
bad_indices_2 = []
i = 2000
for i in range(2000, 7000):
    if i not in bad_indices:
        try:
            transl_kk_sents_goog.append(translateGoogle(cleaned_kazakh_sentences[i], "kk", "en"))
            i += 1
            if i % 200 == 0:
                print("Finished doc at index {}".format(i), flush=True)
        except:
            bad_indices_2.append(i)
            i += 1
            continue
end = time.time()
print(end-start)

Finished doc at index 2200
Finished doc at index 2400
Finished doc at index 2600
Finished doc at index 2800
Finished doc at index 3000
Finished doc at index 3200
Finished doc at index 3400
Finished doc at index 3800
Finished doc at index 4000
Finished doc at index 4400
Finished doc at index 4600
Finished doc at index 4800
Finished doc at index 5000
Finished doc at index 5200
Finished doc at index 5400
Finished doc at index 5600
Finished doc at index 5800
Finished doc at index 6000
Finished doc at index 6200
Finished doc at index 6400
Finished doc at index 6600
Finished doc at index 6800
Finished doc at index 7000
850.3558962345123


In [200]:
print(len(transl_kk_sents_goog)==len(transl_en_sents_goog))

False


In [201]:
subset_sents_en = [cleaned_english_sentences[i] for i in range(2000,7000) if i not in bad_indices]
subset_sents_kk = [cleaned_kazakh_sentences[i] for i in range(2000,7000) if i not in bad_indices]

In [202]:
subset_sents_en_new = []
for doc in subset_sents_en:
    for sent in doc:
        subset_sents_en_new.append(sent)
subset_sents_kk_new = []
for doc in subset_sents_kk:
    for sent in doc:
        subset_sents_kk_new.append(sent)

In [203]:
# Now, we mine in three ways, do majority vote, and take minimum threshold


start = time.time()

i = 0
MOD = 50
all_sent_pairs_en_kk_og, all_margin_scores_en_kk_og = mineSentencePairs(embed(subset_sents_en_new), embed(subset_sents_kk_new), '')
#     all_sent_pairs_en_kk_og.append(doc_sent_pairs)
#     all_margin_scores_en_kk_og.append(doc_margin_scores)
# for en_doc, kk_doc in zip(subset_sents_en, subset_sents_kk):
#     i += 1
#     if i % MOD == 0:
#         print("Completed document {} of {}".format(i, len(subset_sents_en)), flush=True)
#     doc_sent_pairs, doc_margin_scores = mineSentencePairs(embed(en_doc), embed(kk_doc), '')
#     all_sent_pairs_en_kk_og.append(doc_sent_pairs)
#     all_margin_scores_en_kk_og.append(doc_margin_scores)
    
# file_suffix = 'en_kk_old_method'
    
# spf_path = 'Data/sent_pairs_fwd_{:}.csv'.format(file_suffix)
# spb_path = 'Data/sent_pairs_bwd_{:}.csv'.format(file_suffix)
# cct_path = 'Data/concat_pairs_{:}.csv'.format(file_suffix)
# mgn_path = 'Data/margin_scores_{:}.csv'.format(file_suffix)
    
# pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv(spf_path, index=False)
# pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv(spb_path, index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv(cct_path, index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv(mgn_path, index=False)

# all_margin_scores = pd.read_csv(mgn_path)
# all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
# all_sent_pairs = pd.read_csv(cct_path)
# all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
# all_sent_pairs = [pair[0] for pair in all_sent_pairs]

end = time.time()
print("Time taken to mine all sentence pairs: {:.2f} seconds".format(end-start))

Time taken to mine all sentence pairs: 310.05 seconds


In [204]:
# No. of sent pairs mined using original sents
print(len(all_sent_pairs_en_kk_og))

13907


In [205]:
transl_en_sents_goog_new = []
for doc in transl_en_sents_goog:
    for sent in doc:
        transl_en_sents_goog_new.append(sent)
transl_kk_sents_goog_new = []
for doc in transl_kk_sents_goog:
    for sent in doc:
        transl_kk_sents_goog_new.append(sent)

In [206]:
# Now, we mine in three ways, do majority vote, and take minimum threshold


start = time.time()

# i = 0
# MOD = 50
all_sent_pairs_en_to_kk, all_margin_scores_en_to_kk = mineSentencePairs(embed(transl_en_sents_goog_new), embed(subset_sents_kk_new), '')
# for transl_en_doc, kk_doc in zip(transl_en_sents_goog, subset_sents_kk):
#     i += 1
#     if i % MOD == 0:
#         print("Completed document {} of {}".format(i, len(subset_sents_kk)), flush=True)
#     doc_sent_pairs, doc_margin_scores = mineSentencePairs(embed(transl_en_doc), embed(kk_doc), '')
#     all_sent_pairs_en_to_kk.append(doc_sent_pairs)
#     all_margin_scores_en_to_kk.append(doc_margin_scores)
    
# file_suffix = 'en_kk_old_method'
    
# spf_path = 'Data/sent_pairs_fwd_{:}.csv'.format(file_suffix)
# spb_path = 'Data/sent_pairs_bwd_{:}.csv'.format(file_suffix)
# cct_path = 'Data/concat_pairs_{:}.csv'.format(file_suffix)
# mgn_path = 'Data/margin_scores_{:}.csv'.format(file_suffix)
    
# pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv(spf_path, index=False)
# pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv(spb_path, index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv(cct_path, index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv(mgn_path, index=False)

# all_margin_scores = pd.read_csv(mgn_path)
# all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
# all_sent_pairs = pd.read_csv(cct_path)
# all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
# all_sent_pairs = [pair[0] for pair in all_sent_pairs]

end = time.time()
print("Time taken to mine all sentence pairs: {:.2f} seconds".format(end-start))

Time taken to mine all sentence pairs: 317.72 seconds


In [207]:
# Now, we mine in three ways, do majority vote, and take minimum threshold


start = time.time()

# i = 0
# MOD = 50
all_sent_pairs_kk_to_en, all_margin_scores_kk_to_en = mineSentencePairs(embed(subset_sents_en_new), embed(transl_kk_sents_goog_new), '')
# for en_doc, transl_kk_doc in zip(subset_sents_en, transl_kk_sents_goog):
#     i += 1
#     if i % MOD == 0:
#         print("Completed document {} of {}".format(i, len(subset_sents_en)), flush=True)
#     doc_sent_pairs, doc_margin_scores = mineSentencePairs(embed(en_doc), embed(transl_kk_doc), '')
#     all_sent_pairs_kk_to_en.append(doc_sent_pairs)
#     all_margin_scores_kk_to_en.append(doc_margin_scores)
    
# file_suffix = 'en_kk_old_method'
    
# spf_path = 'Data/sent_pairs_fwd_{:}.csv'.format(file_suffix)
# spb_path = 'Data/sent_pairs_bwd_{:}.csv'.format(file_suffix)
# cct_path = 'Data/concat_pairs_{:}.csv'.format(file_suffix)
# mgn_path = 'Data/margin_scores_{:}.csv'.format(file_suffix)
    
# pd.DataFrame({'sent_pairs_fwd': fwd_best}).to_csv(spf_path, index=False)
# pd.DataFrame({'sent_pairs_bwd': bwd_best}).to_csv(spb_path, index=False)
# pd.DataFrame({'concat_pairs': concat_pairs}).to_csv(cct_path, index=False)
# pd.DataFrame({'margin_scores': margin_scores}).to_csv(mgn_path, index=False)

# all_margin_scores = pd.read_csv(mgn_path)
# all_margin_scores = [float(score) for score in all_margin_scores['margin_scores']]
# all_sent_pairs = pd.read_csv(cct_path)
# all_sent_pairs = [[tuple(int(id_) for id_ in pair.strip('()').split(','))] for pair in all_sent_pairs['concat_pairs']]
# all_sent_pairs = [pair[0] for pair in all_sent_pairs]

end = time.time()
print("Time taken to mine all sentence pairs: {:.2f} seconds".format(end-start))

Time taken to mine all sentence pairs: 268.02 seconds


In [223]:
trip1, trip2, trip3 = [], [], []
for p,s in zip(all_sent_pairs_en_kk_og, all_margin_scores_en_kk_og):
    trip1.append((p[0], p[1], s))
for p,s in zip(all_sent_pairs_en_to_kk, all_margin_scores_en_to_kk):
    trip2.append((p[0], p[1], s))
for p,s in zip(all_sent_pairs_kk_to_en, all_margin_scores_kk_to_en):
    trip3.append((p[0], p[1], s))

In [183]:
maj_vote_by_doc = []
for doc1, doc2, doc3 in zip(trip1, trip2, trip3):
    pairs1 = set(doc1)
    pairs2 = set(doc2)
    pairs3 = set(doc3)
    int1,int2,int3 = pairs1&pairs2, pairs1&pairs3, pairs2&pairs3
    majority_vote = list(set(int1|int2|int3))
    maj_vote_by_doc.append(majority_vote)

In [159]:
all_scores = []
for doc in maj_vote_by_doc:
    if len(doc) > 0:
        for score in doc:
            all_scores.append(score[2])

In [163]:
print(np.average(all_scores))

1.2551777978129939


In [221]:
pairs1 = set(all_sent_pairs_en_kk_og)
pairs2 = set(all_sent_pairs_en_to_kk)
pairs3 = set(all_sent_pairs_kk_to_en)
int1,int2,int3 = pairs1&pairs2, pairs1&pairs3, pairs2&pairs3
majority_vote = list(set(int1|int2|int3))

In [222]:
print(len(majority_vote))

6997


In [225]:
all_scores = []
for pair in majority_vote:
    if pair in pairs1:
        all_scores.append(trip1[list(pairs1).index(pair)][2])
    elif pair in pairs2:
        all_scores.append(trip2[list(pairs2).index(pair)][2])
    else:
        all_scores.append(trip3[list(pairs3).index(pair)][2])

In [228]:
print(np.average(all_scores))

1.072879764477287


In [230]:
print(sorted(all_scores)[:10])

[0.9488486647605896, 0.9579350352287292, 0.9591459631919861, 0.9594359993934631, 0.9634297490119934, 0.9634584784507751, 0.9643964171409607, 0.9648369550704956, 0.9649688005447388, 0.9679153561592102]


In [227]:
print(len(all_scores))

6997


In [236]:
en_sents_out = [subset_sents_en_new[pair[0]-1] for pair in majority_vote]

In [237]:
kk_sents_out = [subset_sents_kk_new[pair[1]-1] for pair in majority_vote]

In [238]:
print(len(en_sents_out)==len(kk_sents_out))

True


In [239]:
print(en_sents_out[:10])
print(kk_sents_out[:10])

['Astana won in the nomination of the City of the Future by the National Geographic version.', 'He was elected a Member of the USSR Academy of Sciences in 1958.', "'Yes,' I agreed.", 'The club was founded in 1995.', 'Acquigny Acquigny is a  Communes 20of 20France commune a in the  Eure Eure a  Departments 20of 20France department a in  Normandy Normandy a in northern  France France a .', 'Ak Zhol Democratic Party The Ak Zhol Democratic Party , commonly referred to simply as Ak Zhol , is a  Liberalism liberal a  political 20party political party a in  Kazakhstan Kazakhstan a .', 'Despite this width, the Beluga cannot carry most fuselage parts of the  Airbus 20A380 Airbus A380 a , which are instead normally transported by ship and road.', 'The ancient Greeks used agrimony to treat eye ailments, and it was made into brews for diarrhea and disorders of the gallbladder, liver, and kidneys.', 'The new university would take its name some decades later.', 'Isgandarov got his first education in

In [241]:
majority_vote_subset_pairs = pd.DataFrame({'en': en_sents_out, 'kk': kk_sents_out}).to_csv('Data/en_kk_7k_majority_vote.csv', index=False)

In [252]:
usp = []
for i in range(len(majority_vote)):
    if all_scores[i] > 1.06:
        usp.append((majority_vote[i][0], majority_vote[i][1]))

In [253]:
print(len(usp))

3181


In [254]:
print(usp[:10])

[(69386, 26088), (26050, 8897), (48393, 16823), (46946, 16151), (41847, 8756), (33162, 1345), (31862, 16862), (37659, 12154), (16647, 24768), (64845, 24173)]


In [257]:
en_sents_out_106 = [subset_sents_en_new[pair[0]-1] for pair in usp]
kk_sents_out_106 = [subset_sents_kk_new[pair[1]-1] for pair in usp]

In [258]:
pd.DataFrame({'en': en_sents_out_106, 'kk': kk_sents_out_106}).to_csv('Data/en_kk_7k_majority_vote_1.06.csv', index=False)

In [66]:
# Setting up pipeline for retrieving Tatoeba test data results

def retrieveTatoebaComparisons(lang_id_3, lang_id_2):
    
    start = time.time()
    
    # We'll store the results here and return them at the end
    all_results = []
    
    print("Reading, translating, embedding . . .")
    with open('Data/tatoeba.{}-eng.eng'.format(lang_id_3), 'r') as f:
        en_sents = f.read().splitlines()
    with open('Data/tatoeba.{}-eng.{}'.format(lang_id_3, lang_id_3), 'r') as f:
        xx_sents = f.read().splitlines()
#     en_to_xx_sents = [translateGoogle([en_sent], 'en', lang_id_2)[0] for en_sent in en_sents]
#     xx_to_en_sents = [translateGoogle([xx_sent], lang_id_2, 'en')[0] for xx_sent in xx_sents]
    en_emb = embed(en_sents)
    xx_emb = embed(xx_sents)
#     en_to_xx_emb = embed(en_to_xx_sents)
#     xx_to_en_emb = embed(xx_to_en_sents)
    
    ground_truth_pairs = [(i+1,i+1) for i in range(len(en_sents))]
    
    print("Mining and scoring . . .")
    # METHOD 1: Margin scoring, no threshold
    sent_pairs_og, marg_scores_og = mineSentencePairs(en_emb, xx_emb, '')
    F1, P, R = computeF1(sent_pairs_og, ground_truth_pairs)
    print("F1, P, R for {} with no threshold: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 2: Margin scoring, threshold = 1.06
    above_threshold = []
    THRESHOLD = 1.06
    for i in range(len(sent_pairs_og)):
        if marg_scores_og[i] > THRESHOLD:
            above_threshold.append(sent_pairs_og[i])
    F1, P, R = computeF1(above_threshold, ground_truth_pairs)
    print("F1, P, R for {} with threshold = 1.06: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 3: Margin scoring, threshold = 1.20
    above_threshold = []    
    THRESHOLD = 1.20
    for i in range(len(sent_pairs_og)):
        if marg_scores_og[i] > THRESHOLD:
            above_threshold.append(sent_pairs_og[i])
    F1, P, R = computeF1(above_threshold, ground_truth_pairs)
    print("F1, P, R for {} with threshold = 1.20: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 4: Margin scoring, translation en->xx, no threshold
    sent_pairs_en_xx, marg_scores_en_xx = mineSentencePairs(en_to_xx_emb, xx_emb, '')
    F1, P, R = computeF1(sent_pairs_en_xx, ground_truth_pairs)
    print("F1, P, R for {} using translation en-xx: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 5: Margin scoring, translation xx->en, no threshold
    sent_pairs_xx_en, marg_scores_xx_en = mineSentencePairs(en_emb, xx_to_en_emb, '')
    F1, P, R = computeF1(sent_pairs_xx_en, ground_truth_pairs)
    print("F1, P, R for {} using translation xx_en: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 6: Intersection of methods 1, 4, and 5
    sent_pairs_triple_intersection = list(set(sent_pairs_og)&set(sent_pairs_en_xx)&set(sent_pairs_xx_en))
    F1, P, R = computeF1(sent_pairs_triple_intersection, ground_truth_pairs)
    print("F1, P, R for {} using triple intersection: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    # METHOD 7: Majority vote intersection of methods 1, 4, and 5
    int1 = set(sent_pairs_og)&set(sent_pairs_en_xx)
    int2 = set(sent_pairs_en_xx)&set(sent_pairs_xx_en)
    int3 = set(sent_pairs_og)&set(sent_pairs_xx_en)
    sent_pairs_majority_vote = list(set(int1|int2|int3))
    F1, P, R = computeF1(sent_pairs_majority_vote, ground_truth_pairs)
    print("F1, P, R for {} using majority vote intersection: {}, {}, {}".format(lang_id_3, F1, P, R))
    all_results.append((P, R, F1))
    
    end = time.time()
    print("Time taken: {:.2f} seconds".format(end-start))
    
    return all_results

In [21]:
lang_codes_3c = ['afr', 'amh', 'ang', 'arq', 'arz', 'ast',
                 'awa', 'aze', 'bel', 'ben', 'ber', 'bos',
                 'bre', 'cbk', 'ceb', 'cha', 'cor', 'csb', 
                 'cym', 'dsb', 'dtp', 'epo', 'eus', 'fao', 
                 'fry', 'gla', 'gle', 'gsw', 'hsb', 'ido', 
                 'ile', 'ina', 'isl', 'jav', 'kab', 'kaz', 
                 'khm', 'kur', 'kzj', 'lat', 'lfn', 'mal', 
                 'maz', 'mhr', 'nds', 'nno', 'nov', 'oci', 
                 'orv', 'pam', 'pms', 'swg', 'swh', 'tam', 
                 'tat', 'tel', 'tgl', 'tuk', 'tzl', 'uig', 
                 'uzb', 'war', 'wuu', 'xho', 'yid'] 
lang_codes_2c = ['af', 'am', False, False, False, False, 
                 False, 'az', 'be', 'bn', False, 'bs',
                 False, False, 'ceb', False, 'co', False,
                 'cy', 'pl', False, 'eo', 'eu', False, 
                 'fy', 'gd', 'ga', 'pl', 'eo', 'la',
                 'la', 'is', 'jv', False, 'kk', 'km',
                 'ku', False, 'lv', False, 'mt', False,
                 False, 'de', 'no', False, False, False,
                 False, False, False, 'sw', 'ta', 'tt',
                 'te', 'tl', 'tk', False, 'ug', 'uz',
                 False, False, 'xh', 'yi']

In [64]:
#start = time.time()
results_by_lang = []
for l3,l2 in zip(lang_codes_3c[35:], lang_codes_2c[34:]):
    if l2:
        res = retrieveTatoebaComparisons(l3, l2)
        results_by_lang.append(res)
    else:
        results_by_lang.append((l3, "NULL"))
end = time.time()
print("Total time taken: {:.2f} seconds".format(end-start))

Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for kaz with no threshold: 0.925589831661704, 0.9677419354655077, 0.8869565217391304
F1, P, R for kaz with threshold = 1.06: 0.921532841719058, 0.9692898272366739, 0.8782608695652174
F1, P, R for kaz with threshold = 1.20: 0.8568608045597949, 0.9908675798860532, 0.7547826086956522
F1, P, R for kaz using translation en-xx: 0.9264305127126607, 0.9695817490309965, 0.8869565217391304
F1, P, R for kaz using translation xx_en: 0.9352517935585793, 0.9683426443022655, 0.9043478260869565
F1, P, R for kaz using triple intersection: 0.9116541303624429, 0.9918200408795129, 0.8434782608695652
F1, P, R for kaz using majority vote intersection: 0.9309090859109587, 0.9752380952195192, 0.8904347826086957
Time taken: 275.78 seconds
Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for khm with no threshold: 0.8677013993029753, 0.9302694136144173, 0.8130193905817175
F1, P, R for khm with threshold = 1.06: 0.8644

Mining and scoring . . .
F1, P, R for tgl with no threshold: 0.9863567408268069, 0.9969356486108587, 0.976
F1, P, R for tgl with threshold = 1.06: 0.9842878814632563, 0.9979445015313675, 0.971
F1, P, R for tgl with threshold = 1.20: 0.9333333283505779, 0.9999999999885714, 0.875
F1, P, R for tgl using translation en-xx: 0.9745676450474031, 0.9917184264907689, 0.958
F1, P, R for tgl using translation xx_en: 0.9939698442413626, 0.9989898989798082, 0.989
F1, P, R for tgl using triple intersection: 0.9759344548033262, 0.9999999999895068, 0.953
F1, P, R for tgl using majority vote intersection: 0.9878542460078842, 0.9999999999897541, 0.976
Time taken: 274.97 seconds
Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for tuk with no threshold: 0.8679245233229925, 0.9583333332762897, 0.7931034482758621
F1, P, R for tuk with threshold = 1.06: 0.8729281718453344, 0.9937106917614018, 0.7783251231527094
F1, P, R for tuk with threshold = 1.20: 0.5882352899167875, 0.988372092908

In [69]:
missing_langs = ['ang','arq', 'arz', 'ast', 'awa', 'ber',
                 'bre', 'cbk', 'cha', 'csb', 'dtp', 'fao',
                 'jav', 'kzj','lfn', 'mhr', 'nov', 
                 'oci', 'orv', 'pam', 'pms', 'swg', 'tzl',
                 'war', 'wuu']

In [70]:
#start = time.time()
missing_results = []
for l3 in missing_langs[15:]:
    res = retrieveTatoebaComparisons(l3, '')
    missing_results.append(res)
end = time.time()
print("Total time taken: {:.2f} seconds".format(end-start))

Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for mhr with no threshold: 0.26285713877363265, 0.4599999999885, 0.184
F1, P, R for mhr with threshold = 1.06: 0.26303501600122337, 0.5929824561195445, 0.169
F1, P, R for mhr with threshold = 1.20: 0.18001800000000007, 0.9009009008197387, 0.1
Time taken: 1.33 seconds
Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for nov with no threshold: 0.8541226165839922, 0.9351851851418895, 0.7859922178988327
F1, P, R for nov with threshold = 1.06: 0.82969431820074, 0.9452736317937674, 0.7392996108949417
F1, P, R for nov with threshold = 1.20: 0.7725118435605669, 0.9878787878189164, 0.6342412451361867
Time taken: 0.32 seconds
Reading, translating, embedding . . .
Mining and scoring . . .
F1, P, R for oci with no threshold: 0.7873303118237239, 0.9062499999881999, 0.696
F1, P, R for oci with threshold = 1.06: 0.7766763799732017, 0.9314685314555039, 0.666
F1, P, R for oci with threshold = 1.20: 0.65777184

# Trying majority vote mining on the entire en-kk corpus

In [14]:
# What we'll need for this method:
# (1) The IDs of the sentence pairs we mined using vanilla (en-kk) margin scoring
# (2) The en sentences translated to kk, and the original kk sentences
# (3) The kk sentences translated to en, and the original en sentences
# (4) The document IDs for (2)
# (5) The document IDs for (3)

# Steps:
# 1. Tokenize and embed translated en sentences, then mine with kk sentences
# 2. Repeat step (1) for the translated kk sentences
# 3. Use the majority vote method to retrieve sentence pairs 
#    from the pairwise intersection of of the products of steps (1) 
#    and (2), plus the original sentence pairs

In [30]:
# Reading in (1-5)

# kk sentences translated to en
with open('/project/statnlp/ajones/translation.kk-en', 'r') as f:
    transl_kk_sents = f.read().splitlines()
# original kk sentences
with open('/project/statnlp/ajones/orig.kk-en', 'r') as f:
    orig_kk_sents = f.read().splitlines()
# kk doc IDs
with open('/project/statnlp/ajones/id.kk-en', 'r') as f:
    kk_ids = f.read().splitlines()
    
# en sentences translated to kk
with open('/project/statnlp/ajones/translation.en-kk', 'r') as f:
    transl_en_sents = f.read().splitlines()
# original en sentences
with open('/project/statnlp/ajones/orig.en-kk', 'r') as f:
    orig_en_sents = f.read().splitlines()
# en doc IDs
with open('/project/statnlp/ajones/id.en-kk', 'r') as f:
    en_ids = f.read().splitlines()

In [103]:
orig_en_sents_by_doc = [[] for _ in range(max([literal_eval(id_) for id_ in en_ids]))]

In [104]:
from ast import literal_eval

for i in range(len(orig_en_sents)):
    orig_en_sents_by_doc[literal_eval(en_ids[i])-1].append(orig_en_sents[i])

In [80]:
print(sum([len(d) for d in orig_en_sents_by_doc if len(d) < 30]))

524102


In [79]:
print(len(orig_en_sents_by_doc))

106671


In [81]:
transl_en_sents_by_doc = [[] for _ in range(max([literal_eval(id_) for id_ in en_ids]))]
for i in range(len(transl_en_sents)):
    transl_en_sents_by_doc[literal_eval(en_ids[i])-1].append(transl_en_sents[i])

In [82]:
orig_kk_sents_by_doc = [[] for _ in range(max([literal_eval(id_) for id_ in kk_ids]))]
for i in range(len(orig_kk_sents)):
    orig_kk_sents_by_doc[literal_eval(kk_ids[i])-1].append(orig_kk_sents[i])

In [83]:
transl_kk_sents_by_doc = [[] for _ in range(max([literal_eval(id_) for id_ in kk_ids]))]
for i in range(len(transl_kk_sents)):
    transl_kk_sents_by_doc[literal_eval(kk_ids[i])-1].append(transl_kk_sents[i])

In [85]:
print(len(orig_kk_sents_by_doc)==len(orig_en_sents_by_doc))

True


In [105]:
bad_en_inds = [i for i in range(len(orig_en_sents_by_doc)) if len(orig_en_sents_by_doc[i]) < 30]
bad_kk_inds = [i for i in range(len(orig_kk_sents_by_doc)) if len(orig_kk_sents_by_doc[i]) < 8]

In [107]:
orig_en_sents_by_doc = [orig_en_sents_by_doc[i] for i in range(len(orig_en_sents_by_doc)) 
                        if i not in bad_en_inds and i not in bad_kk_inds]

In [109]:
print(sum([len(d) for d in orig_en_sents_by_doc])) # The number of en sentences after removing very short documents

2200057


In [110]:
orig_kk_sents_by_doc = [orig_kk_sents_by_doc[i] for i in range(len(orig_kk_sents_by_doc)) 
                        if i not in bad_en_inds and i not in bad_kk_inds]

In [111]:
print(sum([len(d) for d in orig_kk_sents_by_doc])) # The number of kk sentences after removing very short documents

410192


In [112]:
transl_en_sents_by_doc = [transl_en_sents_by_doc[i] for i in range(len(transl_en_sents_by_doc)) 
                        if i not in bad_en_inds and i not in bad_kk_inds]
transl_kk_sents_by_doc = [transl_kk_sents_by_doc[i] for i in range(len(transl_kk_sents_by_doc)) 
                        if i not in bad_en_inds and i not in bad_kk_inds]

In [ ]:
# Mining using vanilla margin scoring
all_sent_pairs_en_kk_og, all_margin_scores_en_kk_og = [], []
for en_doc,kk_doc in zip(orig_en_sents_by_doc, orig_kk_sents_by_doc):
    doc_sp, doc_ms = mineSentencePairs(embed(en_doc), embed(kk_doc), '')
    all_sent_pairs_en_kk_og.append(doc_sp)
    all_margin_scores_en_kk_og.append(doc_ms)

In [ ]:
# Mining with en-kk
i = 0
all_sent_pairs_en_to_kk, all_margin_scores_en_to_kk = [], []
for transl_en_doc,kk_doc in zip(transl_en_sents_by_doc, orig_kk_sents_by_doc):
    i += 1
    doc_sp, doc_ms = mineSentencePairs(embed(transl_en_doc), embed(kk_doc), '')
    all_sent_pairs_en_to_kk.append(doc_sp)
    all_margin_scores_en_to_kk.append(doc_ms)
    if i % 100 == 0:
        print("Completed document {} of 14622".format(i))

Completed document 100 of 14622
Completed document 200 of 14622
Completed document 300 of 14622
Completed document 400 of 14622
Completed document 500 of 14622
Completed document 600 of 14622
Completed document 700 of 14622


In [ ]:
# Mining with kk-en
i = 0
all_sent_pairs_kk_to_en, all_margin_scores_kk_to_en = [], []
for en_doc,transl_kk_doc in zip(orig_en_sents_by_doc, transl_kk_sents_by_doc):
    doc_sp, doc_ms = mineSentencePairs(embed(en_doc), embed(transl_kk_doc), '')
    all_sent_pairs_kk_to_en.append(doc_sp)
    all_margin_scores_kk_to_en.append(doc_ms)
    if i % 100 == 0:
        print("Completed document {}".format(i))

In [ ]:
# Taking pairwise intersection

majority_vote_pairs = []
for doc1,doc2,doc3 in zip(all_sent_pairs_en_kk_og,all_sent_pairs_en_to_kk,all_sent_pairs_kk_to_en):
    int1 = set(doc1)&set(doc2)
    int2 = set(doc1)&set(doc3)
    int3 = set(doc2)&set(doc3)
    maj_vote = list(set(int1|int2|int3))
    majority_vote_pairs.append(maj_vote)

In [ ]:
maj_vote_en, maj_vote_kk = [],[]
for i in range(len(majority_vote_pairs)):
    doc = majority_vote_pairs[i]
    if len(doc) > 0:
        for pair in doc:
            maj_vote_en.append(orig_en_sents_by_doc[i][pair[0]-1])
            maj_vote_kk.append(orig_kk_sents_by_doc[i][pair[1]-1])

In [ ]:
pd.DataFrame({'en': maj_vote_en, 'kk': maj_vote_kk}).to_csv('Data/en_kk_mined_majority_vote_ALL.csv', index=False)

In [131]:
print(len(maj_vote_en))

154678


In [151]:
majority_vote_margins = []
for i in range(len(majority_vote_pairs)):
    doc = majority_vote_pairs[i]
    margins = []
    for j in range(len(doc)):
        if doc[j] in all_sent_pairs_en_kk_og[i]:
            ind = all_sent_pairs_en_kk_og[i].index(doc[j])
            margins.append(all_margin_scores_en_kk_og[i][ind])
        elif doc[j] in all_sent_pairs_en_to_kk[i]:
            ind = all_sent_pairs_en_to_kk[i].index(doc[j])
            margins.append(all_margin_scores_en_to_kk[i][ind])
        else:
            ind = all_sent_pairs_kk_to_en[i].index(doc[j])
            margins.append(all_margin_scores_kk_to_en[i][ind])
    majority_vote_margins.append(margins)

In [169]:
i=0
for doc in majority_vote_margins:
    for score in doc:
        if score > 1.35:
            i+=1
print(i)

19099


In [155]:
majority_vote_triples = []
for i in range(len(majority_vote_pairs)):
    doc = majority_vote_pairs[i]
    triples = []
    for j in range(len(doc)):
        if doc[j] in all_sent_pairs_en_kk_og[i]:
            ind = all_sent_pairs_en_kk_og[i].index(doc[j])
            triples.append((doc[j][0], doc[j][1], all_margin_scores_en_kk_og[i][ind]))
        elif doc[j] in all_sent_pairs_en_to_kk[i]:
            ind = all_sent_pairs_en_to_kk[i].index(doc[j])
            triples.append((doc[j][0], doc[j][1], all_margin_scores_en_to_kk[i][ind]))
        else:
            ind = all_sent_pairs_kk_to_en[i].index(doc[j])
            triples.append((doc[j][0], doc[j][1], all_margin_scores_kk_to_en[i][ind]))
    majority_vote_triples.append(triples)

In [170]:
maj_vote_en_135, maj_vote_kk_135 = [],[]
for i in range(len(majority_vote_triples)):
    doc = majority_vote_triples[i]
    for j in range(len(doc)):
        triple = doc[j]
        if triple[2] > 1.35:
            maj_vote_en_135.append(orig_en_sents_by_doc[i][triple[0]-1])
            maj_vote_kk_135.append(orig_kk_sents_by_doc[i][triple[1]-1])

In [171]:
pd.DataFrame({'en': maj_vote_en_135, 'kk': maj_vote_kk_135}).to_csv('Data/en_kk_mined_majority_vote_ALL_1.35.csv', index=False)

In [17]:
en_kk_old_method = pd.read_csv('/project/statnlp/ajones/en_kk_mined_old_method.csv')

In [18]:
en_kk_old_method.head()

,source,target
0,In August 634 the dying Caliph Abu 20Bakr Abu...,халифа Әбу Бәкір Аллаһның оған игілігі мен сәл...
1,This was before the Muslims had entered the ho...,Абдуррахман ибн Ауф Ислам дінін қабылдаған ең ...
2,His name has also been transliterated as Abdel...,Абдуррахман ибн Ауф басқа аты Абдел Рахман ибн...
3,'Abd al Rahman ibn 'Awf 'Abd al Rahman ibn 'Aw...,"Абдуррахман ибн Ауф Абдуррахман ибн Ауф, көрне..."
4,He was buried on a hill to the north east of p...,Хиджраның жылы Осман ибн Аффан ның билігі кезі...


In [19]:
en_kk_old_method.shape

(430761, 2)

In [22]:
en_sents, kk_sents = list(en_kk_old_method['source']), list(en_kk_old_method['target'])

In [27]:
kk_sents = [t for t in kk_sents if type(t)==str]

In [28]:
en_embs, kk_embs = embed(en_sents), embed(kk_sents)

In [51]:
old_sent_pairs, old_margin_scores = mineSentencePairs(en_embs, kk_embs, 'en_kk_old_method_v2')

Retrieving means in forward direction
Time taken to retrieve forward means: 3976.96 seconds
Retrieving means in backward direction
Completed pair 310000 of 429327 in the backward direction
Completed pair 320000 of 429327 in the backward direction
Completed pair 330000 of 429327 in the backward direction
Completed pair 340000 of 429327 in the backward direction
Completed pair 350000 of 429327 in the backward direction
Completed pair 360000 of 429327 in the backward direction
Completed pair 370000 of 429327 in the backward direction
Completed pair 380000 of 429327 in the backward direction
Completed pair 390000 of 429327 in the backward direction
Completed pair 400000 of 429327 in the backward direction
Completed pair 410000 of 429327 in the backward direction
Completed pair 420000 of 429327 in the backward direction
Mining in the backward direction complete


In [53]:
usp = []
for i in range(len(old_sent_pairs)):
    if old_margin_scores[i] > 1.35:
            usp.append(old_sent_pairs[i])

In [54]:
len(usp)

2745